Lets Begin Coding
1. Download and Install required modules and libraries
2. Import required data and connect with drive


In [ ]:

!pip install lpips
!pip install bcolz #bcolz 1.2.0
!pip install torch torchvision
#!pip install mxnet-cu90 #optional, for data processing MXNet 1.3.1
!pip install focal-loss-torch
!pip install split-folders # Used for split folders into train and val datasets or in our case split data into gallery and probe
!pip install PTable
#https://torchmetrics.readthedocs.io/en/latest/image/learned_perceptual_image_patch_similarity.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 28.9 MB/s 
  Created wheel for bcolz: filename=bcolz-1.2.1-cp38-cp38-linux_x86_64.whl size=2873144 sha256=aa017152b1dafc8987b58fb1556ffeeb63c191032b392558e8445fdc1fc8402d
  Stored in directory: /root/.cache/pip/wheels/c8/f3/3b/5cf5f3997e4729700f17eb4faa8eb7dde1a209cdea66371b7a
Successfully built bcolz
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from PIL import Image
import numpy as np
import lpips
import torchvision.transforms as transforms
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
# used to supress display of warnings
import warnings
from sklearn.metrics import precision_recall_curve,accuracy_score,f1_score,precision_score,recall_score

import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import os

#from tensorboardX import SummaryWriter
from tqdm import tqdm

#4
import cv2
import torchvision.datasets as datasets
import torch.nn.functional as F
from tqdm import tqdm

#5
import torch.utils.data as data
import torch.nn.functional as F

#66
from easydict import EasyDict as edict
from pathlib import Path
from torch.nn import CrossEntropyLoss
from torchvision import transforms as trans
from torch.utils.data import Dataset, ConcatDataset, DataLoader
from torchvision import transforms as trans
from torchvision.datasets import ImageFolder
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import numpy as np
import cv2
import bcolz
import pickle


import splitfolders
from torch import nn, optim, as_tensor
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
import cv2
from PIL import Image
from pdb import set_trace
import time
import copy
from pathlib import Path
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage import io, transform
from tqdm import trange, tqdm
import csv
import glob
import dlib
import pandas as pd
import numpy as np
from prettytable import PrettyTable
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Lowkey code/supp_material'

/content/drive/MyDrive/Lowkey code/supp_material


In [ ]:
%ls

 align/               imgattacked2.png         pnet.npy
 attack1.png          imgattacked.png          requirements.txt
 attack_dir_warp.py   imgattackshalwar.JPG     rnet.npy
 backbone/           'Lowkey test run.ipynb'   util/
 face/                models/
 flagged/             onet.npy


In [ ]:
#convert to tensor
to_tensor = transforms.ToTensor()

A little info about folders
1. Align,util, models and backbone are imported from lowkey. Align is a copy of similar folder from FaceEvolve.
2. Face is a clone of complete repository of FaceEvolve.

In [ ]:
import os
import sys
import math
import numbers

In [ ]:
from util.prepare_utils import prepare_models,prepare_dir_vec,get_ensemble,extract_features # Used to load attack backbone in code
from util.feature_extraction_utils import normalize_transforms,feature_extractor
from util.attack_utils import Attack
from align.detector import detect_faces
from align.align_trans import get_reference_facial_points,warp_and_crop_face
from backbone.model_irse import IR_152
import lpips
from face.applications.align.visualization_utils import show_results

#1

from face.head.metrics import ArcFace,CosFace,SphereFace
from face.util.utils import make_weights_for_balanced_classes, get_val_data,separate_irse_bn_paras, separate_resnet_bn_paras, warm_up_lr, schedule_lr, perform_val, get_time, buffer_val, AverageMeter, accuracy

#2

from backbone.model_irse import IR_50, IR_101, IR_152, IR_SE_50, IR_SE_101, IR_SE_152
from backbone.model_resnet import ResNet_50, ResNet_101, ResNet_152

from face.loss.focal import FocalLoss

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
device

device(type='cuda')

 Parameters for lowkey adversial Attack

In [ ]:
# Parameters for lowkey adversial Attack
eps = 0.05
n_iters = 50 # No of epooch for the attack
input_size = [112, 112] # Input size model expects
attack_type = 'lpips' # attack type used
c_tv = None
c_sim = 0.05
lr = 0.0025 # the learning rate to optimize attack on
net_type = 'alex'
noise_size = 0.005 # noise size used
n_starts = 1
kernel_size_gf = 7 # gaussian smoothing window
sigma_gf = 3 # sigma for gaussain smoothing
combination = True # A switch to use gaussain smoothing together in attack or not
using_subspace = False
V_reduction_root = './'
direction = 1
crop_size = 112
scale = crop_size / 112 #1


Parameters for training by Face Evolve.

In [ ]:

EMBEDDING_SIZE = 512 # feature dimension
BATCH_SIZE = 512
DROP_LAST = True # whether drop the last batch to ensure consistent batch_norm statistics
LR = 0.1 # initial LR
NUM_EPOCH = 120 # total epoch number (use the firt 1/25 epochs to warm up)
WEIGHT_DECAY = 5e-4 # do not apply to batch_norm parameters
MOMENTUM = 0.9
STAGES = [35, 65, 95] # epoch stages to decay learning rate
HEAD_NAME = 'ArcFace' # support:  ['Softmax', 'ArcFace', 'CosFace', 'SphereFace', 'Am_softmax']
LOSS_NAME = 'Focal' # support: ['Focal', 'Softmax']

#INPUT_SIZE = [112, 112] # support: [112, 112] and [224, 224]
RGB_MEAN = [0.5, 0.5, 0.5] # for normalize inputs to [-1, 1]

#DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
MULTI_GPU = True # flag to use multiple GPUs; if you choose to train with single GPU, you should first run "export CUDA_VISILE_DEVICES=device_id" to specify the GPU card you want to use
GPU_ID = [0] # specify your GPU ids
PIN_MEMORY = True
NUM_WORKERS = 0

RGB_MEAN = [0.5, 0.5, 0.5] # for normalize inputs to [-1, 1]
RGB_STD = [0.5, 0.5, 0.5]

### 1. Load all the **Models**

---



In [ ]:
model_backbones = ['IR_152', 'IR_152', 'ResNet_152', 'ResNet_152'] # Model backbone for ensemble
model_roots = ['models/Backbone_IR_152_Arcface_Epoch_112.pth', 'models/Backbone_IR_152_Cosface_Epoch_70.pth', \
'models/Backbone_ResNet_152_Arcface_Epoch_65.pth', 'models/Backbone_ResNet_152_Cosface_Epoch_68.pth'] # Roots of the models


In [ ]:
# Load an IR_152 Cosface model
PATH = 'models/Backbone_IR_152_Cosface_Epoch_70.pth'
IR_152C = IR_152(input_size=input_size)
IR_152C.load_state_dict(torch.load(PATH))

# Load IR_152 Arc Face model
PATH1 = 'models/Backbone_IR_152_Arcface_Epoch_112.pth'
IR_152A = IR_152(input_size=input_size)
IR_152A.load_state_dict(torch.load(PATH1))

#Load ResNet 152 Arc Face Model
PATH2 = 'models/Backbone_ResNet_152_Arcface_Epoch_65.pth'
RESNET_152A = ResNet_152(input_size=input_size)
RESNET_152A.load_state_dict(torch.load(PATH2))


# Load Resnet 152 CosFace Model
PATH3 = 'models/Backbone_ResNet_152_Cosface_Epoch_68.pth'
RESNET_152C = ResNet_152(input_size=input_size)
RESNET_152C.load_state_dict(torch.load(PATH3))



<All keys matched successfully>

In [ ]:
# Generate lowkey ensemble attack model
models_attack, V_reduction, dim = prepare_models(model_backbones,
              input_size,
              model_roots,
              kernel_size_gf,
              sigma_gf,
              combination,
              using_subspace,
              V_reduction_root)

Loading Attack Backbone Checkpoint '['models/Backbone_IR_152_Arcface_Epoch_112.pth', 'models/Backbone_IR_152_Cosface_Epoch_70.pth', 'models/Backbone_ResNet_152_Arcface_Epoch_65.pth', 'models/Backbone_ResNet_152_Cosface_Epoch_68.pth']'


### 2. Now lets perform evaluation on clean data gallery - 24 identities

---





In [ ]:
# define data transform
transform = transforms.Compose([
      transforms.Resize([int(128 * input_size[0] / 112), int(128 * input_size[0] / 112)]),  # smaller side resized
      transforms.CenterCrop([input_size[0], input_size[1]]),
      transforms.ToTensor(),
      transforms.Normalize(mean = RGB_MEAN, std = RGB_STD)])



In [ ]:
#Define Loaders for all different galleries and corresponding probe

# Define the Loader for Clean data gallery and probe and the adversial infected gallery.

gallery_root = '/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/gallery' #90% of validation data
probe_root = '/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/probe' # 10% of validation data
adversial_gallery_root = '/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble' #90% of validation data
#Load Datasetsfor clean gallery and adversial infected gallery

gallery_dataset = datasets.ImageFolder(gallery_root, transform)
probe_dataset = datasets.ImageFolder(probe_root, transform)
adversial_gallery_dataset = datasets.ImageFolder(adversial_gallery_root, transform)
# Define Data Loaders
gallery_loader = torch.utils.data.DataLoader(gallery_dataset, batch_size = 16, shuffle = False, pin_memory = True, num_workers = 0)
probe_loader = torch.utils.data.DataLoader(probe_dataset, batch_size = 16, shuffle = False, pin_memory = True, num_workers = 0)
adversial_loader = torch.utils.data.DataLoader(adversial_gallery_dataset, batch_size = 16, shuffle = False, pin_memory = True, num_workers = 0)

In [ ]:
print(len(gallery_dataset))

1240


In [ ]:
print(len(adversial_gallery_dataset))

1240


In [ ]:
print(len(probe_dataset))

151


In [ ]:
gallery_dataset

Dataset ImageFolder
    Number of datapoints: 1240
    Root location: /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/gallery
    StandardTransform
Transform: Compose(
               Resize(size=[128, 128], interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=[112, 112])
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [ ]:
probe_dataset

Dataset ImageFolder
    Number of datapoints: 151
    Root location: /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/probe
    StandardTransform
Transform: Compose(
               Resize(size=[128, 128], interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=[112, 112])
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [ ]:
adversial_gallery_dataset

Dataset ImageFolder
    Number of datapoints: 1240
    Root location: /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble
    StandardTransform
Transform: Compose(
               Resize(size=[128, 128], interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=[112, 112])
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [ ]:
# Methods to Process features of the data for face recognition
def l2_norm(input, axis = 1):
    norm = torch.norm(input, 2, axis, True)
    output = torch.div(input, norm)

    return output


def de_preprocess(tensor):

    return tensor * 0.5 + 0.5


hflip = transforms.Compose([
            de_preprocess,
            transforms.ToPILImage(),
            transforms.functional.hflip,
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])


def hflip_batch(imgs_tensor):
    hfliped_imgs = torch.empty_like(imgs_tensor)
    for i, img_ten in enumerate(imgs_tensor):
        hfliped_imgs[i] = hflip(img_ten)

    return hfliped_imgs


In [ ]:
# Define a function for feature extractor

def extract_feature_face(loader, model, embedding_size = 512, batch_size = 16, device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"), tta = True):

    NUM_CLASS = len(loader.dataset.classes)
    print("Number of Classes: {}".format(NUM_CLASS))

    model.to(device)

    # extract features
    model.eval() # set to evaluation mode
    idx = 0
    features = np.zeros([len(loader.dataset), embedding_size])
    print("=" * 60)
    print("Features zeros shape: {}".format(features.shape))

    with torch.no_grad():

        iter_loader = iter(loader)
        while idx + batch_size <= len(loader.dataset):
            print("=" * 60)
            print("idx + batch_size <= len(loader.dataset)")
            batch, _ = next(iter_loader)
            features[idx:idx + batch_size] = l2_norm(model(batch.to(device))).cpu()
            print("=" * 60)
            print("Features l2_norm shape else : {}".format(features.shape))
            idx += batch_size
            print("The value of idx is : {}".format(idx))

        if idx < len(loader.dataset):
            batch, _ = next(iter_loader)
            print("=" * 60)
            print("idx<len now")
            features[idx:] = l2_norm(model(batch.to(device)).cpu())
            print("=" * 60)
            print("Features l2_norm else shape : {}".format(features.shape))

    images = []
    labels = []
    for img_path, _ in loader.dataset.samples:
        img = cv2.imread(img_path)
        images.append(img_path)
        label = _
        label = labels.append(label)
    return labels,images, features

In [ ]:
# #Load gallery,probe and labels for IR-152 ArcFace model

gallery_features_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/gallery1_features/gallery_IR_152A_features.npy")
gallery_labels_IR_152A =np.load("/content/drive/MyDrive/Lowkey code/gallery1_features/gallery_labels_IR_152A.npy")
probe_features_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/probe1_features/probe_IR_152A_features.npy")
probe_labels_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/probe1_features/probe_labels_IR_152A.npy")

# #Load gallery,probe and labels for IR-152 CosFace model

gallery_features_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/gallery1_features/gallery_features_IR_152C.npy")
gallery_labels_IR_152C =np.load("/content/drive/MyDrive/Lowkey code/gallery1_features/gallery_labels_IR_152C.npy")
probe_features_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/probe1_features/probe_features_IR_152C.npy")
probe_labels_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/probe1_features/probe_labels_IR_152C.npy")

# #Load gallery,probe and labels for RESNET-152 ArcFace model

gallery_features_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/gallery1_features/gallery_features_RESNET_152A.npy")
gallery_labels_RESNET_152A =np.load("/content/drive/MyDrive/Lowkey code/gallery1_features/gallery_labels_RESNET_152A.npy")
probe_features_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/probe1_features/probe_features_RESNET_152A.npy")
probe_labels_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/probe1_features/probe_labels_RESNET_152A.npy")

# #Load gallery,probe and labels for RESNET-152 CosFace model

gallery_features_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/gallery1_features/gallery_features_RESNET_152C.npy")
gallery_labels_RESNET_152C =np.load("/content/drive/MyDrive/Lowkey code/gallery1_features/gallery_labels_RESNET_152C.npy")
probe_features_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/probe1_features/probe_features_RESNET_152C.npy")
probe_labels_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/probe1_features/probe_labels_RESNET_152C.npy")


In [ ]:
from numpy import dot, sqrt

def cosine_similarity(x, y):
    return dot(x, y) / (sqrt(dot(x, x)) * sqrt(dot(y, y)))

In [ ]:
# print("img 1 is: {} ".format(gallery_images_IR_152A[0]))
# print("img 2 is: {} ".format(gallery_images_IR_152A[50]))

score_IR152A = cosine_similarity(gallery_features_IR_152A[0],gallery_features_IR_152A[50])
print("IR_152 ArcFace similarity score is : {}".format(score_IR152A))

score_IR152C = cosine_similarity(gallery_features_IR_152C[0],gallery_features_IR_152C[50])
print("IR_152 CosFace similarity score is : {}".format(score_IR152C))

score_RES152A = cosine_similarity(gallery_features_RESNET_152A[0],gallery_features_RESNET_152A[50])
print("RESNET_152 ArcFace similarity score is : {}".format(score_RES152A))

score_RES152C = cosine_similarity(gallery_features_RESNET_152C[0],gallery_features_RESNET_152C[50])
print("RESNET_152 CosFace similarity score is : {}".format(score_RES152C))

IR_152 ArcFace similarity score is : 0.7643686561941939
IR_152 CosFace similarity score is : 0.6357569693547866
RESNET_152 ArcFace similarity score is : 0.6490438878541562
RESNET_152 CosFace similarity score is : 0.656360885412436


### 3. Lets define a function that uses gallery and probe features to compute top1 and top 5 accuracy


---



In [ ]:
import heapq
heapq.nlargest(3,[1,2,3,4,5])

[5, 4, 3]

In [ ]:
gallery_labels_IR_152A[100:120]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
probe_labels_IR_152A[:3]

array([0, 0, 0])

In [ ]:
from numpy import dot, sqrt

def get_best_match(source,target,real_label,all_labels,n=1):
  "takes source feature and target features, along with source label and target labels. Returns TRUE if the label of source exists in top N labels of targets"
  sims=[]
  for val in target:
    cos = cosine_similarity(source,val)
    sims.append(cos) # calculate cosine similarity with all

  vals=heapq.nlargest(n,sims) # n largest similarity values
  indices=[]
  possible_labels=[]
  for val in vals:
    ind=sims.index(val) # index of largest similarity
    indices.append(ind)
    possible_labels.append(all_labels[ind]) # get label corresponding to best matches
  if real_label in possible_labels: # the given value exists in the top N similarities
    return True
  return False

get_best_match(probe_features_IR_152A[0],gallery_features_IR_152A,probe_labels_IR_152A[0],gallery_labels_IR_152A,n=3)

True

In [ ]:
get_best_match(probe_features_IR_152A[100],gallery_features_IR_152A[:99],probe_labels_IR_152A[100],gallery_labels_IR_152A[:99])

False

In [ ]:
# Compute top 1 and top 5 accuracy of model and generate the evaluation table as lowkey here
def get_accuracy(probe_features,probe_labels,gallery_features,gallery_labels,k=1):
  correct=0
  with tqdm(total=len(probe_features)) as pbar:
    for i,val in enumerate(probe_features):
      flag=get_best_match(val,gallery_features,probe_labels[i],gallery_labels,n=k)
      if flag:
        correct+=1
      pbar.update(1)
  accuracy = correct/len(probe_features)
  #print("Top 1 accuracy of gallery is ", correct/len(probe_features))

  return  accuracy


In [ ]:
# calculate accuracy for IR_152A
clean_top1_accuracy_IR152A = get_accuracy(probe_features=probe_features_IR_152A,probe_labels=probe_labels_IR_152A,
                                          gallery_features=gallery_features_IR_152A,gallery_labels=gallery_labels_IR_152A,k=1)

# calculate accuracy for IR_152C
clean_top1_accuracy_IR152C = get_accuracy(probe_features=probe_features_IR_152C,probe_labels=probe_labels_IR_152C,
                                          gallery_features=gallery_features_IR_152C,gallery_labels=gallery_labels_IR_152C,k=1)

clean_top1_accuracy_RESNET152A = get_accuracy(probe_features=probe_features_RESNET_152A,probe_labels=probe_labels_RESNET_152A,
                                          gallery_features=gallery_features_RESNET_152A,gallery_labels=gallery_labels_RESNET_152A,k=1)

clean_top1_accuracy_RESNET152C = get_accuracy(probe_features=probe_features_RESNET_152C,probe_labels=probe_labels_RESNET_152C,
                                          gallery_features=gallery_features_RESNET_152C,gallery_labels=gallery_labels_RESNET_152C,k=1)




100%|██████████| 151/151 [00:01<00:00, 100.54it/s]


In [ ]:
# calculate accuracy for IR_152A
clean_top5_accuracy_IR152A = get_accuracy(probe_features=probe_features_IR_152A,probe_labels=probe_labels_IR_152A,
                                          gallery_features=gallery_features_IR_152A,gallery_labels=gallery_labels_IR_152A,k=5)

# calculate accuracy for IR_152C
clean_top5_accuracy_IR152C = get_accuracy(probe_features=probe_features_IR_152C,probe_labels=probe_labels_IR_152C,
                                          gallery_features=gallery_features_IR_152C,gallery_labels=gallery_labels_IR_152C,k=5)

clean_top5_accuracy_RESNET152A = get_accuracy(probe_features=probe_features_RESNET_152A,probe_labels=probe_labels_RESNET_152A,
                                          gallery_features=gallery_features_RESNET_152A,gallery_labels=gallery_labels_RESNET_152A,k=5)

clean_top5_accuracy_RESNET152C = get_accuracy(probe_features=probe_features_RESNET_152C,probe_labels=probe_labels_RESNET_152C,
                                          gallery_features=gallery_features_RESNET_152C,gallery_labels=gallery_labels_RESNET_152C,k=5)




100%|██████████| 151/151 [00:01<00:00, 101.03it/s]


In [ ]:
print("The top1 Accuracy on clean images for IR_152A is: {}".format(clean_top1_accuracy_IR152A))
print("The top1 Accuracy on clean images for IR_152C is: {}".format(clean_top1_accuracy_IR152C))
print("The top1 Accuracy on clean images for RESNET_152A is: {}".format(clean_top1_accuracy_RESNET152A))
print("The top1 Accuracy on clean images for RESNET_152C is: {}".format(clean_top1_accuracy_RESNET152C))


The top1 Accuracy on clean images for IR_152A is: 0.9933774834437086
The top1 Accuracy on clean images for IR_152C is: 0.9933774834437086
The top1 Accuracy on clean images for RESNET_152A is: 0.9933774834437086
The top1 Accuracy on clean images for RESNET_152C is: 0.9933774834437086


In [ ]:
print("The top5 Accuracy on clean images for IR_152A is: {}".format(clean_top5_accuracy_IR152A))
print("The top5 Accuracy on clean images for IR_152C is: {}".format(clean_top5_accuracy_IR152C))
print("The top5 Accuracy on clean images for RESNET_152A is: {}".format(clean_top5_accuracy_RESNET152A))
print("The top5 Accuracy on clean images for RESNET_152C is: {}".format(clean_top5_accuracy_RESNET152C))


The top5 Accuracy on clean images for IR_152A is: 0.9933774834437086
The top5 Accuracy on clean images for IR_152C is: 0.9933774834437086
The top5 Accuracy on clean images for RESNET_152A is: 0.9933774834437086
The top5 Accuracy on clean images for RESNET_152C is: 1.0


### 4. Now Lets perform the attack on 5 identities and insert them back to gallery to create adversial gallery


---




In [ ]:
# Function to protect a single image

def protect_image(image):
  #img = Image.fromarray(img)
  img = Image.open(image)
  #print(img)
  reference = get_reference_facial_points(default_square=True) * scale
  print("The value for reference in get reference facial points  is {}".format(reference))
  h,w,c = np.array(img).shape
  print (h,w,c)

  b_box,landmarks = detect_faces(img)
  #show_results(img, _, landmarks)
  print("The size of landmarks is {}".format(landmarks.size))
  #print("The value for _ in detect faces is {}".format(_))
  # (landmarks)
  facial5points = [[landmarks[0][j], landmarks[0][j + 5]] for j in range(5)]
  #print("The value for _ in detect faces is {} end ".format(facial5points))
  b_box, tfm = warp_and_crop_face(np.array(img), facial5points, reference, crop_size=(crop_size, crop_size))
  print("The value for tfm in warp_and_crop_face is {}  ".format(tfm))
  # pytorch transform
  theta = normalize_transforms(tfm, w, h)
  tensor_img = to_tensor(img).unsqueeze(0).to(device)
  print("The value for theta in normalize_transforms(tfm, w, h) is {}  ".format(theta))
  V_reduction = None
  dim = 512

  # Find gradient direction vector
  dir_vec_extractor = get_ensemble(models = models_attack, sigma_gf=None, kernel_size_gf=None, combination=False, V_reduction=V_reduction, warp=True, theta_warp=theta)
  dir_vec = prepare_dir_vec(dir_vec_extractor, tensor_img, dim, combination)
  print("The value for direction vector of gradient in image is {}  ".format(dir_vec))
  img_attacked = tensor_img.clone()
  attack = Attack(models_attack, dim, attack_type,
                  eps, c_sim, net_type, lr, n_iters,
                  noise_size, n_starts, c_tv, sigma_gf,
                  kernel_size_gf, combination, warp=True,
                  theta_warp=theta, V_reduction=V_reduction)
  img_attacked = attack.execute(tensor_img, dir_vec, direction).detach().cpu()

  img_attacked_pil = transforms.ToPILImage()(img_attacked[0])
  return img_attacked_pil,dir_vec_extractor


In [ ]:
#Now lets create a  function to attack a directory of images  with lowkey attack

def protect_dir(dirs_root,model,save_dir):

  for img_name in tqdm(os.listdir(dirs_root)):


    img_root = os.path.join(dirs_root, img_name)
    #new_img_root = os.path.join(save_dir, img_name)
    print("The image processed is : {}".format(img_root))
    print('Finding reference points')
    reference = get_reference_facial_points(default_square=True) * scale
    img = Image.open(img_root)
    #h,w,c = np.array(img).shape
    h,w,c = np.array(img).shape
    #print("The value for reference in get reference facial points  is {}".format(reference))
    #print (h,w,c)
    ## Detects facial points ##
    b_box,landmarks = detect_faces(img)
    facial5points = [[landmarks[0][j], landmarks[0][j + 5]] for j in range(5)]
    #print("The value for _ in detect faces is {} end ".format(facial5points))
    b_box, tfm = warp_and_crop_face(np.array(img), facial5points, reference, crop_size=(crop_size, crop_size))
    #print("The value for tfm in warp_and_crop_face is {}  ".format(tfm))

    ## find pytorch transform
    theta = normalize_transforms(tfm, w, h)
    tensor_img = to_tensor(img).unsqueeze(0).to(device)
    #print("The value for theta in normalize_transforms(tfm, w, h) is {}  ".format(theta))

    V_reduction = None
    dim = 512

  # Find gradient direction vector i.e find the feature vector extractor
    dir_vec_extractor = get_ensemble(models=model, sigma_gf=None, kernel_size_gf=None, combination=False, V_reduction=V_reduction, warp=True, theta_warp=theta)
    dir_vec = prepare_dir_vec(dir_vec_extractor, tensor_img, dim, combination).to(device)
  #print("The value for direction vector of gradient in image is {}  ".format(dir_vec))

  # Perform Low key attack
    img_attacked = tensor_img.clone()
    attack = Attack(model, dim, attack_type,
                  eps, c_sim, net_type, lr, n_iters,
                  noise_size, n_starts, c_tv, sigma_gf,
                  kernel_size_gf, combination, warp=True,
                  theta_warp=theta, V_reduction=V_reduction).to(device)
    img_attacked = attack.execute(tensor_img, dir_vec, direction).detach().cpu()
    print("attack Executed")
    img_attacked_pil = transforms.ToPILImage()(img_attacked[0])
    img_attacked_pil.save(img_root[:-4] + 'protected.png')
  #numimg += 1
  #print ("Attack succesful on :" +numimg)
    #return dir_vec_extractor

#Return the feature extractor

In [ ]:
# Now onto adversial attack on some identities
lowkey_id_1 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Adam_Brody"
lowkey_id_2 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/AndyGarcia"
lowkey_id_3 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Andy_Serkis"
lowkey_id_4 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Ben_Affleck"
lowkey_id_5 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Alfred_Molina"



In [ ]:
# protect_dir(dirs_root=lowkey_id_3,model=models_attack)


  0%|          | 0/58 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Andy_Serkis/Andy_Serkis_5918_3439.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  2%|▏         | 1/58 [00:36<34:26, 36.26s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Andy_Serkis/Andy_Serkis_5896_3424.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  3%|▎         | 2/58 [01:13<34:32, 37.00s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Andy_Serkis/Andy_Serkis_5904_3430.protected.png
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 98%|█████████▊| 57/58 [35:15<00:36, 36.47s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Andy_Serkis/Andy_Serkis_5911_3435.protected.png
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 58/58 [35:51<00:00, 37.10s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=lowkey_id_4,model=models_attack)


  0%|          | 0/27 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Ben_Affleck/Ben_Affleck_8549_4727.jpeg
Finding reference points


/content/drive/MyDrive/Lowkey code/supp_material/align/first_stage.py:32: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img = Variable(torch.FloatTensor(_preprocess(img)), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:81: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img_boxes = Variable(torch.FloatTensor(img_boxes), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:115: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:102: UserWarning: volatile was remov

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Ben_Affleck/Ben_Affleck_8461_4662.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 96%|█████████▋| 26/27 [16:03<00:36, 36.09s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Ben_Affleck/Ben_Affleck_8476_4672.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 27/27 [16:38<00:00, 37.00s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=lowkey_id_5,model=models_attack)


  0%|          | 0/17 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Alfred_Molina/Alfred_Molina_4109_2483.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  6%|▌         | 1/17 [00:35<09:27, 35.48s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Alfred_Molina/Alfred_Molina_3944_2366.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 12%|█▏        | 2/17 [01:12<09:08, 36.55s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Alfred_Molina/Alfred_Molina_3994_2404.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 94%|█████████▍| 16/17 [09:38<00:36, 36.71s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Alfred_Molina/Alfred_Molina_3952_2372.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 17/17 [10:14<00:00, 36.17s/it]

attack Executed


In [ ]:
# # from matplotlib.patches import Arc
# # Extract adversial gallery features for IR_152A
# adversial_labels_IR_152A,adversial_images_IR_152A,adversial_features_IR_152A = extract_feature_face(loader=adversial_loader,model=IR_152A,embedding_size=EMBEDDING_SIZE,device=device)

# np.save("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_features_IR_152A.npy", adversial_features_IR_152A)
# np.save("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_labels_IR_152A.npy", adversial_labels_IR_152A)
# np.save("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_images_IR_152A.npy", adversial_images_IR_152A)

# #Extract adversial gallery features for IR_152C
# adversial_labels_IR_152C,adversial_images_IR_152C,adversial_features_IR_152C = extract_feature_face(loader=adversial_loader,model=IR_152C,embedding_size=EMBEDDING_SIZE,device=device)

# np.save("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_features_IR_152C.npy", adversial_features_IR_152C)
# np.save("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_labels_IR_152C.npy", adversial_labels_IR_152C)
# np.save("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_images_IR_152C.npy", adversial_images_IR_152C)


# #Extract adversial gallery features for  RESNET_152A
# adversial_labels_RESNET_152A,adversial_images_RESNET_152A,adversial_features_RESNET_152A = extract_feature_face(loader=adversial_loader,model=RESNET_152A,embedding_size=EMBEDDING_SIZE,device=device)

# np.save("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_features_RESNET_152A.npy", adversial_features_RESNET_152A)
# np.save("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_labels_RESNET_152A.npy", adversial_labels_RESNET_152A)
# np.save("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_images_RESNET_152A.npy", adversial_images_RESNET_152A)

# #Extract adversial gallery features for RESNET_152C
# adversial_labels_RESNET_152C,adversial_images_RESNET_152C,adversial_features_RESNET_152C = extract_feature_face(loader=adversial_loader,model=RESNET_152C,embedding_size=EMBEDDING_SIZE,device=device)

# np.save("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_features_RESNET_152C.npy", adversial_features_RESNET_152C)
# np.save("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_labels_RESNET_152C.npy", adversial_labels_RESNET_152C)
# np.save("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_images_RESNET_152C.npy", adversial_images_RESNET_152C)


Number of Classes: 24
Features zeros shape: (1240, 512)
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 16
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 32
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 48
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 64
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 80
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 96
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 112
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 128
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The 

In [ ]:
adversial_features_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_features_IR_152A.npy")
adversial_labels_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_labels_IR_152A.npy")

adversial_features_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_features_IR_152C.npy")
adversial_labels_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_labels_IR_152C.npy")

adversial_features_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_features_RESNET_152A.npy")
adversial_labels_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_labels_RESNET_152A.npy")

adversial_features_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_features_RESNET_152C.npy")
adversial_labels_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_labels_RESNET_152C.npy")


In [ ]:
type(gallery_features_IR_152A)

numpy.ndarray

In [ ]:
type(probe_features_IR_152A)

numpy.ndarray

In [ ]:
type(adversial_features_IR_152A)

numpy.ndarray

In [ ]:
# calculate accuracy for IR_152A
adversial_top1_accuracy_IR152A = get_accuracy(probe_features=probe_features_IR_152A,probe_labels=probe_labels_IR_152A,
                                          gallery_features=adversial_features_IR_152A,gallery_labels=adversial_labels_IR_152A,k=1)

# calculate accuracy for IR_152C
adversial_top1_accuracy_IR152C = get_accuracy(probe_features=probe_features_IR_152C,probe_labels=probe_labels_IR_152C,
                                          gallery_features=adversial_features_IR_152C,gallery_labels=adversial_labels_IR_152C,k=1)

adversial_top1_accuracy_RESNET152A = get_accuracy(probe_features=probe_features_RESNET_152A,probe_labels=probe_labels_RESNET_152A,
                                          gallery_features=adversial_features_RESNET_152A,gallery_labels=adversial_labels_RESNET_152A,k=1)

adversial_top1_accuracy_RESNET152C = get_accuracy(probe_features=probe_features_RESNET_152C,probe_labels=probe_labels_RESNET_152C,
                                          gallery_features=adversial_features_RESNET_152C,gallery_labels=adversial_labels_RESNET_152C,k=1)



100%|██████████| 151/151 [00:01<00:00, 104.18it/s]


In [ ]:
# calculate accuracy for IR_152A
adversial_top5_accuracy_IR152A = get_accuracy(probe_features=probe_features_IR_152A,probe_labels=probe_labels_IR_152A,
                                          gallery_features=adversial_features_IR_152A,gallery_labels=adversial_labels_IR_152A,k=5)

# calculate accuracy for IR_152C
adversial_top5_accuracy_IR152C = get_accuracy(probe_features=probe_features_IR_152C,probe_labels=probe_labels_IR_152C,
                                          gallery_features=adversial_features_IR_152C,gallery_labels=adversial_labels_IR_152C,k=5)

adversial_top5_accuracy_RESNET152A = get_accuracy(probe_features=probe_features_RESNET_152A,probe_labels=probe_labels_RESNET_152A,
                                          gallery_features=adversial_features_RESNET_152A,gallery_labels=adversial_labels_RESNET_152A,k=5)

adversial_top5_accuracy_RESNET152C = get_accuracy(probe_features=probe_features_RESNET_152C,probe_labels=probe_labels_RESNET_152C,
                                          gallery_features=adversial_features_RESNET_152C,gallery_labels=adversial_labels_RESNET_152C,k=5)



100%|██████████| 151/151 [00:01<00:00, 102.53it/s]


In [ ]:
print("The top1 Accuracy on adversial images for IR_152A is: {}".format(adversial_top1_accuracy_IR152A))
print("The top1 Accuracy on adversial images for IR_152C is: {}".format(adversial_top1_accuracy_IR152C))
print("The top1 Accuracy on adversial images for RESNET_152A is: {}".format(adversial_top1_accuracy_RESNET152A))
print("The top1 Accuracy on adversial images for RESNET_152C is: {}".format(adversial_top1_accuracy_RESNET152C))


The top1 Accuracy on adversial images for IR_152A is: 0.7880794701986755
The top1 Accuracy on adversial images for IR_152C is: 0.7947019867549668
The top1 Accuracy on adversial images for RESNET_152A is: 0.7947019867549668
The top1 Accuracy on adversial images for RESNET_152C is: 0.7947019867549668


In [ ]:
print("The top5 Accuracy on adversial images for IR_152A is: {}".format(adversial_top5_accuracy_IR152A))
print("The top5 Accuracy on adversial images for IR_152C is: {}".format(adversial_top5_accuracy_IR152C))
print("The top5 Accuracy on adversial images for RESNET_152A is: {}".format(adversial_top5_accuracy_RESNET152A))
print("The top5 Accuracy on adversial images for RESNET_152C is: {}".format(adversial_top5_accuracy_RESNET152C))


The top5 Accuracy on adversial images for IR_152A is: 0.7947019867549668
The top5 Accuracy on adversial images for IR_152C is: 0.7947019867549668
The top5 Accuracy on adversial images for RESNET_152A is: 0.7947019867549668
The top5 Accuracy on adversial images for RESNET_152C is: 0.8013245033112583


In [ ]:
gallery_images_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/gallery1_features/gallery_images_IR_152A.npy")
probe_images_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/probe1_features/probe_images_IR_152A.npy")
adversial_images_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/adversial1_features/adversial_images_IR_152A.npy")

In [ ]:
print("img 1 is: {} ".format(gallery_images_IR_152A[0]))
print("img 2 is: {} ".format(probe_images_IR_152A[1]))

score_IR152A = cosine_similarity(gallery_features_IR_152A[0],probe_features_IR_152A[1])
print("IR_152 ArcFace score is : {}".format(score_IR152A))

score_IR152C = cosine_similarity(gallery_features_IR_152C[0],probe_features_IR_152C[1])
print("IR_152 CosFace score is : {}".format(score_IR152C))

score_RES152A = cosine_similarity(gallery_features_RESNET_152A[0],probe_features_RESNET_152A[1])
print("RESNET_152 ArcFace score is : {}".format(score_RES152A))

score_RES152C = cosine_similarity(gallery_features_RESNET_152C[0],probe_features_RESNET_152C[1])
print("RESNET_152 CosFace score is : {}".format(score_RES152C))

img 1 is: /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/gallery/Aaron_Eckhart/Aaron_Eckhart_105_83.jpeg 
img 2 is: /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/probe/Aaron_Eckhart/Aaron_Eckhart_257_176.jpeg 
IR_152 ArcFace score is : 0.7413190771717567
IR_152 CosFace score is : 0.6754996214791377
RESNET_152 ArcFace score is : 0.7050748316227949
RESNET_152 CosFace score is : 0.7014041057537382


In [ ]:
print("img 1 is: {} ".format(adversial_images_IR_152A[0]))
print("img 2 is: {} ".format(probe_images_IR_152A[1]))

score_IR152A = cosine_similarity(adversial_features_IR_152A[0],probe_features_IR_152A[1])
print("IR_152 ArcFace score is : {}".format(score_IR152A))

score_IR152C = cosine_similarity(adversial_features_IR_152C[0],probe_features_IR_152C[1])
print("IR_152 CosFace score is : {}".format(score_IR152C))

score_RES152A = cosine_similarity(adversial_features_RESNET_152A[0],probe_features_RESNET_152A[1])
print("RESNET_152 ArcFace score is : {}".format(score_RES152A))

score_RES152C = cosine_similarity(adversial_features_RESNET_152C[0],probe_features_RESNET_152C[1])
print("RESNET_152 CosFace score is : {}".format(score_RES152C))

img 1 is: /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_galleries/adversial_gallery_ensemble/Aaron_Eckhart/Aaron_Eckhart_105_83.jpeg 
img 2 is: /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/probe/Aaron_Eckhart/Aaron_Eckhart_257_176.jpeg 
IR_152 ArcFace score is : 0.7413190771717567
IR_152 CosFace score is : 0.6754996214791377
RESNET_152 ArcFace score is : 0.7050748316227949
RESNET_152 CosFace score is : 0.7014041057537382


In [ ]:

tableEvaluation = PrettyTable()

# Add data for table display

tableEvaluation.title = 'Evaluation pipeline Top 1'
tableEvaluation.field_names = ['Attacker','IR_152A', 'IR_152C','RESNET_152A','RESNET_152C']
tableEvaluation.add_row(['clean_images_gallery',clean_top1_accuracy_IR152A*100,clean_top1_accuracy_IR152C*100,clean_top1_accuracy_RESNET152A*100,clean_top1_accuracy_RESNET152C*100])
tableEvaluation.add_row(['Ensemble_attack_gallery',adversial_top1_accuracy_IR152A*100,adversial_top1_accuracy_IR152C*100, adversial_top1_accuracy_RESNET152A*100,adversial_top1_accuracy_RESNET152C*100])

print(tableEvaluation)


+---------------------------------------------------------------------------------------------------------+
|                                        Evaluation pipeline Top 1                                        |
+-------------------------+-------------------+-------------------+-------------------+-------------------+
|         Attacker        |      IR_152A      |      IR_152C      |    RESNET_152A    |    RESNET_152C    |
+-------------------------+-------------------+-------------------+-------------------+-------------------+
|   clean_images_gallery  | 99.33774834437085 | 99.33774834437085 | 99.33774834437085 | 99.33774834437085 |
| Ensemble_attack_gallery | 78.80794701986756 | 79.47019867549669 | 79.47019867549669 | 79.47019867549669 |
+-------------------------+-------------------+-------------------+-------------------+-------------------+


In [ ]:

tableEvaluation2 = PrettyTable()

# Add data for table display

tableEvaluation2.title = 'Evaluation pipeline Top 5'
tableEvaluation2.field_names = ['Attacker','IR_152A', 'IR_152C','RESNET_152A','RESNET_152C']
tableEvaluation2.add_row(['clean_images_gallery', clean_top5_accuracy_IR152A*100,clean_top5_accuracy_IR152C*100,clean_top5_accuracy_RESNET152A*100,clean_top5_accuracy_RESNET152C*100])
tableEvaluation2.add_row(['Ensemble_attack_gallery',adversial_top5_accuracy_IR152A*100,adversial_top5_accuracy_IR152C*100, adversial_top5_accuracy_RESNET152A*100,adversial_top5_accuracy_RESNET152C*100])

print(tableEvaluation2)


+---------------------------------------------------------------------------------------------------------+
|                                        Evaluation pipeline Top 5                                        |
+-------------------------+-------------------+-------------------+-------------------+-------------------+
|         Attacker        |      IR_152A      |      IR_152C      |    RESNET_152A    |    RESNET_152C    |
+-------------------------+-------------------+-------------------+-------------------+-------------------+
|   clean_images_gallery  | 99.33774834437085 | 99.33774834437085 | 99.33774834437085 |       100.0       |
| Ensemble_attack_gallery | 79.47019867549669 | 79.47019867549669 | 79.47019867549669 | 80.13245033112582 |
+-------------------------+-------------------+-------------------+-------------------+-------------------+


### 5. Now Lets Check accuracy by attacking 5 identities and computing accuracy against their corresponding probe for all models


---



In [ ]:
# Okay now lets check the accuracy w.r.t to only adversial gallery and its corresponding probe
lowkey_gallery_root = '/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery' #90% of validation data
lowkey_probe_root = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe"

# Generate Dataset
lowkey_gallery_dataset = datasets.ImageFolder(lowkey_gallery_root, transform)
lowkey_probe_dataset = datasets.ImageFolder(lowkey_probe_root, transform)

# Generate loader with batch size = 16
lowkey_gallery_loader = torch.utils.data.DataLoader(lowkey_gallery_dataset, batch_size = 16, shuffle = False, pin_memory = True, num_workers = 0)
lowkey_probe_loader = torch.utils.data.DataLoader(lowkey_probe_dataset, batch_size = 16, shuffle = False, pin_memory = True, num_workers = 0)

In [ ]:
#Lets make loader for the corresponding probes for all attack
#Probe features with IR_152
all_attack_probe_labels_IR_152A,all_attack_probe_images_IR_152A,all_attack_probe_features_IR_152A = extract_feature_face(loader=lowkey_probe_loader,model=IR_152A,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_labels_IR_152A.npy",all_attack_probe_labels_IR_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_images_IR_152A.npy",all_attack_probe_images_IR_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_features_IR_152A.npy",all_attack_probe_features_IR_152A)

#Probe features with IR_152C
all_attack_probe_labels_IR_152C,all_attack_probe_images_IR_152C,all_attack_probe_features_IR_152C = extract_feature_face(loader=lowkey_probe_loader,model=IR_152C,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_labels_IR_152C.npy",all_attack_probe_labels_IR_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_images_IR_152C.npy",all_attack_probe_images_IR_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_features_IR_152C.npy",all_attack_probe_features_IR_152C)

#Probe features with RESNET_152A
all_attack_probe_labels_RESNET_152A,all_attack_probe_images_RESNET_152A,all_attack_probe_features_RESNET_152A = extract_feature_face(loader=lowkey_probe_loader,model=RESNET_152A,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_labels_RESNET_152A.npy",all_attack_probe_labels_RESNET_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_images_RESNET_152A.npy",all_attack_probe_images_RESNET_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_features_RESNET_152A.npy",all_attack_probe_features_RESNET_152A)

#Probe features with RESNET_152C
all_attack_probe_labels_RESNET_152C,all_attack_probe_images_RESNET_152C,all_attack_probe_features_RESNET_152C = extract_feature_face(loader=lowkey_probe_loader,model=RESNET_152C,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_labels_RESNET_152C.npy",all_attack_probe_labels_RESNET_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_images_RESNET_152C.npy",all_attack_probe_images_RESNET_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_features_RESNET_152C.npy",all_attack_probe_features_RESNET_152C)



Number of Classes: 5
Features zeros shape: (35, 512)
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (35, 512)
The value of idx is : 16
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (35, 512)
The value of idx is : 32
idx<len now
Features l2_norm else shape : (35, 512)
Number of Classes: 5
Features zeros shape: (35, 512)
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (35, 512)
The value of idx is : 16
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (35, 512)
The value of idx is : 32
idx<len now
Features l2_norm else shape : (35, 512)
Number of Classes: 5
Features zeros shape: (35, 512)
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (35, 512)
The value of idx is : 16
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (35, 512)
The value of idx is : 32
idx<len now
Features l2_norm else shape : (35, 512)
Number of Classes: 5
Features zeros shape: (35, 512)
id

In [ ]:
# #Lets do it for new Work
# #Load labels for IR_152A
# all_attack_probe_labels_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_labels_IR_152A.npy")
# all_attack_probe_features_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_features_IR_152A.npy")

# #Load labels for IR_152C
# all_attack_probe_labels_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_labels_IR_152C.npy")
# all_attack_probe_features_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_features_IR_152C.npy")

# #Load labels for RESNET_152A
# all_attack_labels_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_labels_RESNET_152A.npy")
# all_attack_features_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_features_RESNET_152A.npy")

# #Load labels for RESNET_152C
# all_attack_labels_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_labels_RESNET_152C.npy")
# all_attack_features_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/probe_features/all_attack_probe_features_RESNET_152C.npy")


In [ ]:
# Generate Ensemble attacked Gallery features
from matplotlib.patches import Arc
#Lets extract features for IR_152A
lowkey_gallery_labels_IR_152A,lowkey_gallery_images_IR_152A,lowkey_gallery_features_IR_152A = extract_feature_face(loader=lowkey_gallery_loader,model=IR_152A,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_labels_IR_152A.npy", lowkey_gallery_labels_IR_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_images_IR_152A.npy", lowkey_gallery_images_IR_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_features_IR_152A.npy", lowkey_gallery_features_IR_152A)

#Extract gallery features for IR_152C
lowkey_gallery_labels_IR_152C,lowkey_gallery_images_IR_152C,lowkey_gallery_features_IR_152C = extract_feature_face(loader=lowkey_gallery_loader,model=IR_152C,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_labels_IR_152C.npy", lowkey_gallery_labels_IR_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_images_IR_152C.npy", lowkey_gallery_images_IR_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_features_IR_152C.npy", lowkey_gallery_features_IR_152C)

#Extract gallery features for  RESNET_152A
lowkey_gallery_labels_RESNET_152A,lowkey_gallery_images_RESNET_152A,lowkey_gallery_features_RESNET_152A = extract_feature_face(loader=lowkey_gallery_loader,model=RESNET_152A,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_labels_RESNET_152A.npy", lowkey_gallery_labels_RESNET_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_images_RESNET_152A.npy", lowkey_gallery_images_RESNET_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_features_RESNET_152A.npy", lowkey_gallery_features_RESNET_152A)

#extract gallery features for RESNET_152C
lowkey_gallery_labels_RESNET_152C,lowkey_gallery_images_RESNET_152C,lowkey_gallery_features_RESNET_152C = extract_feature_face(loader=lowkey_gallery_loader,model=RESNET_152C,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_labels_RESNET_152C.npy", lowkey_gallery_labels_RESNET_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_images_RESNET_152C.npy", lowkey_gallery_images_RESNET_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_features_RESNET_152C.npy", lowkey_gallery_features_RESNET_152C)





Number of Classes: 24
Features zeros shape: (1240, 512)
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 16
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 32
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 48
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 64
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 80
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 96
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 112
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The value of idx is : 128
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1240, 512)
The 

In [ ]:
# #Lets do it for new Work
# #Load labels for IR_152A
# lowkey_gallery_labels_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_labels_IR_152A.npy")
# lowkey_gallery_features_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_features_IR_152A.npy")

# #Load labels for IR_152C
# lowkey_gallery_labels_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_labels_IR_152C.npy")
# lowkey_gallery_features_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_features_IR_152C.npy")

# #Load labels for RESNET_152A
# lowkey_gallery_labels_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_labels_RESNET_152A.npy")
# lowkey_gallery_features_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_features_RESNET_152A.npy")

# #Load labels for RESNET_152C
# lowkey_gallery_labels_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_labels_RESNET_152C.npy")
# lowkey_gallery_features_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/Ensemble_attack/gallery_features/lowkey_gallery_features_RESNET_152C.npy")


In [ ]:
# now lets calculate the lowkey accuracies
# calculate accuracy for IR_152A
lowkey_top1_accuracy_IR152A = get_accuracy(probe_features=all_attack_probe_features_IR_152A,
                                               probe_labels=all_attack_probe_labels_IR_152A,
                                          gallery_features=lowkey_gallery_features_IR_152A,
                                          gallery_labels=lowkey_gallery_labels_IR_152A,k=1)

# calculate accuracy for IR_152C
lowkey_top1_accuracy_IR152C = get_accuracy(probe_features=all_attack_probe_features_IR_152C,
                                               probe_labels=all_attack_probe_labels_IR_152C,
                                          gallery_features=lowkey_gallery_features_IR_152C,
                                          gallery_labels=lowkey_gallery_labels_IR_152C,k=1)

# calculate accuracy for RESNET_152C
lowkey_top1_accuracy_RESNET152A = get_accuracy(probe_features=all_attack_probe_features_RESNET_152A,
                                               probe_labels=all_attack_probe_labels_RESNET_152A,
                                          gallery_features=lowkey_gallery_features_RESNET_152A,
                                          gallery_labels=lowkey_gallery_labels_RESNET_152A,k=1)


# calculate accuracy for RESNET_152C
lowkey_top1_accuracy_RESNET152C = get_accuracy(probe_features=all_attack_probe_features_RESNET_152C,
                                               probe_labels=all_attack_probe_labels_RESNET_152C,
                                          gallery_features=lowkey_gallery_features_RESNET_152C,
                                          gallery_labels=lowkey_gallery_labels_RESNET_152C,k=1)



100%|██████████| 35/35 [00:00<00:00, 101.34it/s]


In [ ]:
print("The top1 Accuracy on lowkey images for IR_152A is: {}%".format(lowkey_top1_accuracy_IR152A *100))
print("The top1 Accuracy on lowkey images for IR_152C is: {}%".format(lowkey_top1_accuracy_IR152C *100))
print("The top1 Accuracy on lowkey images for RESNET_152A is: {}%".format(lowkey_top1_accuracy_RESNET152A *100))
print("The top1 Accuracy on lowkey images for RESNET_152C is: {}%".format(lowkey_top1_accuracy_RESNET152A *100))


The top1 Accuracy on lowkey images for IR_152A is: 0.0%
The top1 Accuracy on lowkey images for IR_152C is: 0.0%
The top1 Accuracy on lowkey images for RESNET_152A is: 0.0%
The top1 Accuracy on lowkey images for RESNET_152C is: 0.0%


In [ ]:
# now lets calculate the lowkey accuracies
# calculate accuracy for IR_152A
lowkey_top5_accuracy_IR152A = get_accuracy(probe_features=all_attack_probe_features_IR_152A,
                                               probe_labels=all_attack_probe_labels_IR_152A,
                                          gallery_features=lowkey_gallery_features_IR_152A,
                                          gallery_labels=lowkey_gallery_labels_IR_152A,k=5)

# calculate accuracy for IR_152C
lowkey_top5_accuracy_IR152C = get_accuracy(probe_features=all_attack_probe_features_IR_152C,
                                               probe_labels=all_attack_probe_labels_IR_152C,
                                          gallery_features=lowkey_gallery_features_IR_152C,
                                          gallery_labels=lowkey_gallery_labels_IR_152C,k=5)

# calculate accuracy for RESNET_152C
lowkey_top5_accuracy_RESNET152A = get_accuracy(probe_features=all_attack_probe_features_RESNET_152A,
                                               probe_labels=all_attack_probe_labels_RESNET_152A,
                                          gallery_features=lowkey_gallery_features_RESNET_152A,
                                          gallery_labels=lowkey_gallery_labels_RESNET_152A,k=5)


# calculate accuracy for RESNET_152C

lowkey_top5_accuracy_RESNET152C = get_accuracy(probe_features=all_attack_probe_features_RESNET_152C,
                                               probe_labels=all_attack_probe_labels_RESNET_152C,
                                          gallery_features=lowkey_gallery_features_RESNET_152C,
                                          gallery_labels=lowkey_gallery_labels_RESNET_152C,k=5)



100%|██████████| 35/35 [00:00<00:00, 98.76it/s]


In [ ]:
print("The top5 Accuracy on lowkey images for IR_152A is: {}%".format(lowkey_top5_accuracy_IR152A *100))
print("The top5 Accuracy on lowkey images for IR_152C is: {}%".format(lowkey_top5_accuracy_IR152C *100))
print("The top5 Accuracy on lowkey images for RESNET_152A is: {}%".format(lowkey_top5_accuracy_RESNET152A *100))
print("The top5 Accuracy on lowkey images for RESNET_152C is: {}%".format(lowkey_top5_accuracy_RESNET152A *100))


The top5 Accuracy on lowkey images for IR_152A is: 2.857142857142857%
The top5 Accuracy on lowkey images for IR_152C is: 5.714285714285714%
The top5 Accuracy on lowkey images for RESNET_152A is: 14.285714285714285%
The top5 Accuracy on lowkey images for RESNET_152C is: 14.285714285714285%


In [ ]:

tableEvaluation4 = PrettyTable()

# Add data for table display

tableEvaluation4.title = 'Evaluation pipeline Top 1'
tableEvaluation4.field_names = ['Attacker','IR_152A', 'IR_152C','RESNET_152A','RESNET_152C']
tableEvaluation4.add_row(['clean_Gallery', clean_top1_accuracy_IR152A*100,clean_top1_accuracy_IR152C*100,clean_top1_accuracy_RESNET152A*100,clean_top1_accuracy_RESNET152C*100])
tableEvaluation4.add_row(['Adversial_Gallery',adversial_top1_accuracy_IR152A*100,adversial_top1_accuracy_IR152C*100, adversial_top1_accuracy_RESNET152A*100,adversial_top1_accuracy_RESNET152C*100])
tableEvaluation4.add_row(['Lowkey_attack_5',lowkey_top1_accuracy_IR152A *100,lowkey_top1_accuracy_IR152C *100, lowkey_top1_accuracy_RESNET152A *100,lowkey_top1_accuracy_RESNET152A *100])

print(tableEvaluation4)


+---------------------------------------------------------------------------------------------------+
|                                     Evaluation pipeline Top 1                                     |
+-------------------+-------------------+-------------------+-------------------+-------------------+
|      Attacker     |      IR_152A      |      IR_152C      |    RESNET_152A    |    RESNET_152C    |
+-------------------+-------------------+-------------------+-------------------+-------------------+
|   clean_Gallery   | 99.33774834437085 | 99.33774834437085 | 99.33774834437085 | 99.33774834437085 |
| Adversial_Gallery | 78.80794701986756 | 79.47019867549669 | 79.47019867549669 | 79.47019867549669 |
|  Lowkey_attack_5  |        0.0        |        0.0        |        0.0        |        0.0        |
+-------------------+-------------------+-------------------+-------------------+-------------------+


In [ ]:

tableEvaluation3 = PrettyTable()

# Add data for table display

tableEvaluation3.title = 'Evaluation pipeline Top 5'
tableEvaluation3.field_names = ['Attacker','IR_152A', 'IR_152C','RESNET_152A','RESNET_152C']
tableEvaluation3.add_row(['clean_gallery', clean_top5_accuracy_IR152A*100,clean_top5_accuracy_IR152C*100,clean_top5_accuracy_RESNET152A*100,clean_top5_accuracy_RESNET152C*100])
tableEvaluation3.add_row(['Adversial_gallery',adversial_top5_accuracy_IR152A*100,adversial_top5_accuracy_IR152C*100, adversial_top5_accuracy_RESNET152A*100,adversial_top5_accuracy_RESNET152C*100])
tableEvaluation3.add_row(['Lowkey_attack_5',lowkey_top5_accuracy_IR152A *100,lowkey_top5_accuracy_IR152C *100, lowkey_top5_accuracy_RESNET152A *100,lowkey_top5_accuracy_RESNET152A *100])

print(tableEvaluation3)


+-----------------------------------------------------------------------------------------------------+
|                                      Evaluation pipeline Top 5                                      |
+-------------------+-------------------+-------------------+--------------------+--------------------+
|      Attacker     |      IR_152A      |      IR_152C      |    RESNET_152A     |    RESNET_152C     |
+-------------------+-------------------+-------------------+--------------------+--------------------+
|   clean_gallery   | 99.33774834437085 | 99.33774834437085 | 99.33774834437085  |       100.0        |
| Adversial_gallery | 79.47019867549669 | 79.47019867549669 | 79.47019867549669  | 80.13245033112582  |
|  Lowkey_attack_5  | 2.857142857142857 | 5.714285714285714 | 14.285714285714285 | 14.285714285714285 |
+-------------------+-------------------+-------------------+--------------------+--------------------+


### 6. Now Lets Perform attack on gallery with  IR_152 Arc Face model


---



In [ ]:
IR_152_attack_id_1 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Adam_Brody"
IR_152_attack_id_2 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Alfred_Molina"
IR_152_attack_id_3 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Andy_Garcia"
IR_152_attack_id_4 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Andy_Serkis"
IR_152_attack_id_5 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Ben_Affleck"



In [ ]:
#Create model lists to be used with Lowkey protect_dir folder as Lowkey takes models in form of lists
model_IR_152A_list =[]
model_IR_152C_list =[]
model_RESNET_152A_list =[]
model_RESNET_152C_list =[]

model_IR_152A_list.append(IR_152A)
model_IR_152C_list.append(IR_152C)
model_RESNET_152A_list.append(RESNET_152A)
model_RESNET_152C_list.append(RESNET_152C)

In [ ]:
# #Lets attack the directories with IR_152ArcFace of models to compute adversial attack
# protect_dir(dirs_root=IR_152_attack_id_1,model=model_IR_152A_list)

  0%|          | 0/69 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:4289: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:4227: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Adam_Brody/Adam_Brody_700_427.jpeg
Finding reference points


/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Adam_Brody/Adam_Brody_428_283.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 99%|█████████▊| 68/69 [10:21<00:09,  9.63s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Adam_Brody/Adam_Brody_411_269.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 69/69 [10:30<00:00,  9.14s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=IR_152_attack_id_2,model=model_IR_152A_list)

  0%|          | 0/43 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Alfred_Molina/Alfred_Molina_4224_2548.jpeg
Finding reference points


/content/drive/MyDrive/Lowkey code/supp_material/align/first_stage.py:32: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img = Variable(torch.FloatTensor(_preprocess(img)), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:81: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img_boxes = Variable(torch.FloatTensor(img_boxes), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:115: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:102: UserWarning: volatile was remov

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  2%|▏         | 1/43 [00:09<06:27,  9.23s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Alfred_Molina/Alfred_Molina_4007_2415.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 40%|███▉      | 17/43 [02:35<03:54,  9.02s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Alfred_Molina/Alfred_Molina_3996_2406.jpeg
Finding reference points


 40%|███▉      | 17/43 [02:36<03:59,  9.23s/it]


IndexError: ignored

In [ ]:
# protect_dir(dirs_root=IR_152_attack_id_2,model=model_IR_152A_list)

  0%|          | 0/25 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Alfred_Molina/Alfred_Molina_3959_2378.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  4%|▍         | 1/25 [00:09<03:36,  9.04s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Alfred_Molina/Alfred_Molina_4078_2467.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  8%|▊         | 2/25 [00:17<03:24,  8.90s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Alfred_Molina/Alfred_Molina_3952_2372.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 96%|█████████▌| 24/25 [03:36<00:08,  8.92s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Alfred_Molina/Alfred_Molina_3944_2366.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 25/25 [03:45<00:00,  9.03s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=IR_152_attack_id_3,model=model_IR_152A_list)

  0%|          | 0/69 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Andy_Garcia/Andy_Garcia_5271_3034.jpeg
Finding reference points


/content/drive/MyDrive/Lowkey code/supp_material/align/first_stage.py:32: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img = Variable(torch.FloatTensor(_preprocess(img)), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:81: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img_boxes = Variable(torch.FloatTensor(img_boxes), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:115: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:102: UserWarning: volatile was remov

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  1%|▏         | 1/69 [00:10<11:25, 10.09s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Andy_Garcia/Andy_Garcia_4981_2809.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 65%|██████▌   | 45/69 [06:52<03:39,  9.15s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Andy_Garcia/Andy_Garcia_5079_2891.jpeg
Finding reference points


 65%|██████▌   | 45/69 [06:53<03:40,  9.18s/it]


IndexError: ignored

In [ ]:
# protect_dir(dirs_root=IR_152_attack_id_3,model=model_IR_152A_list)

  0%|          | 0/23 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Andy_Garcia/Andy_Garcia_5048_2867.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  4%|▍         | 1/23 [00:09<03:23,  9.27s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Andy_Garcia/Andy_Garcia_5034_2853.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  9%|▊         | 2/23 [00:18<03:12,  9.17s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Andy_Garcia/Andy_Garcia_4960_2790.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 96%|█████████▌| 22/23 [03:20<00:09,  9.21s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Andy_Garcia/Andy_Garcia_4959_2789.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 23/23 [03:29<00:00,  9.11s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=IR_152_attack_id_4,model=model_IR_152A_list)

  0%|          | 0/50 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Andy_Serkis/Andy_Serkis_5844_3393.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  2%|▏         | 1/50 [00:08<07:16,  8.91s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Andy_Serkis/Andy_Serkis_5935_3444.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  4%|▍         | 2/50 [00:18<07:14,  9.05s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Andy_Serkis/Andy_Serkis_5652_3287.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 98%|█████████▊| 49/50 [07:24<00:08,  8.97s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Andy_Serkis/Andy_Serkis_5640_3278.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 50/50 [07:33<00:00,  9.07s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=IR_152_attack_id_5,model=model_IR_152A_list)

  0%|          | 0/64 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Ben_Affleck/Ben_Affleck_8812_4907.jpeg
Finding reference points


/content/drive/MyDrive/Lowkey code/supp_material/align/first_stage.py:32: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img = Variable(torch.FloatTensor(_preprocess(img)), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:81: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img_boxes = Variable(torch.FloatTensor(img_boxes), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:115: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:102: UserWarning: volatile was remov

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Ben_Affleck/Ben_Affleck_8455_4656.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 98%|█████████▊| 63/64 [09:36<00:09,  9.26s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery/Ben_Affleck/Ben_Affleck_8471_4670.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 64/64 [09:46<00:00,  9.16s/it]

attack Executed


In [ ]:
# Now lets define a loader for IR_152_attacked gallery
IR_152A_attack_gallery_root = '/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery' #90% of validation data
# Generate Dataset
IR_152A_attack_gallery_dataset = datasets.ImageFolder(IR_152A_attack_gallery_root, transform)
# Generate loader with batch size = 16
IR_152A_attack_gallery_loader = torch.utils.data.DataLoader(IR_152A_attack_gallery_dataset, batch_size = 16, shuffle = False, pin_memory = True, num_workers = 0)


In [ ]:
#Lets generate the features for IR_152A_attacked identities
from matplotlib.patches import Arc
#Lets extract features for IR_152A
attack_IR152A_labels_IR_152A,attack_IR152A_images_IR_152A,attack_IR152A_features_IR_152A = extract_feature_face(loader=IR_152A_attack_gallery_loader,model=IR_152A,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__labels_IR_152A.npy", attack_IR152A_labels_IR_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__images_IR_152A.npy", attack_IR152A_images_IR_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__features_IR_152A.npy", attack_IR152A_features_IR_152A)

#Lets extract features for IR_152C
attack_IR152A_labels_IR_152C,attack_IR152A_images_IR_152C,attack_IR152A_features_IR_152C = extract_feature_face(loader=IR_152A_attack_gallery_loader,model=IR_152C,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__labels_IR_152C.npy", attack_IR152A_labels_IR_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__images_IR_152C.npy", attack_IR152A_images_IR_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__features_IR_152C.npy", attack_IR152A_features_IR_152C)

#Lets extract features for RESNET_152A
attack_IR152A_labels_RESNET_152A,attack_IR152A_images_RESNET_152A,attack_IR152A_features_RESNET_152A = extract_feature_face(loader=IR_152A_attack_gallery_loader,model=RESNET_152A,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__labels_RESNET_152A.npy", attack_IR152A_labels_RESNET_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__images_RESNET_152A.npy", attack_IR152A_images_RESNET_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__features_RESNET_152A.npy", attack_IR152A_features_RESNET_152A)

#extract gallery features for RESNET_152C

attack_IR152A_labels_RESNET_152C,attack_IR152A_images_RESNET_152C,attack_IR152A_features_RESNET_152C = extract_feature_face(loader=IR_152A_attack_gallery_loader,model=RESNET_152C,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__labels_RESNET_152C.npy", attack_IR152A_labels_RESNET_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__images_RESNET_152C.npy", attack_IR152A_images_RESNET_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__features_RESNET_152C.npy", attack_IR152A_features_RESNET_152C)




Number of Classes: 24
Features zeros shape: (1243, 512)
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 16
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 32
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 48
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 64
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 80
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 96
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 112
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 128
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The 

In [ ]:
#Lets create the loader for these directories
attack_IR152A_labels_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__labels_IR_152A.npy")
attack_IR152A_features_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__features_IR_152A.npy")

attack_IR152A_labels_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__labels_IR_152C.npy")
attack_IR152A_features_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__features_IR_152C.npy")


attack_IR152A_labels_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__labels_RESNET_152A.npy")
attack_IR152A_features_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__features_RESNET_152A.npy")

attack_IR152A_labels_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__labels_RESNET_152C.npy")
attack_IR152A_features_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152A_attack/gallery_features/attack_IR152A__features_RESNET_152C.npy")


In [ ]:
# Lets calculate top1 accuracy
# now lets calculate the lowkey accuracies
# calculate accuracy for IR_152A
IR_152A_attack_top1_accuracy_IR152A = get_accuracy(probe_features=all_attack_probe_features_IR_152A,
                                               probe_labels=all_attack_probe_labels_IR_152A,
                                          gallery_features=attack_IR152A_features_IR_152A,
                                          gallery_labels=attack_IR152A_labels_IR_152A,k=1)

# calculate accuracy for IR_152C
IR_152A_attack_top1_accuracy_IR152C = get_accuracy(probe_features=all_attack_probe_features_IR_152C,
                                               probe_labels=all_attack_probe_labels_IR_152C,
                                          gallery_features=attack_IR152A_features_IR_152C,
                                          gallery_labels=attack_IR152A_labels_IR_152C,k=1)

# calculate accuracy for RESNET_152C
IR_152A_attack_top1_accuracy_RESNET152A = get_accuracy(probe_features=all_attack_probe_features_RESNET_152A,
                                               probe_labels=all_attack_probe_labels_RESNET_152A,
                                          gallery_features=attack_IR152A_features_RESNET_152A,
                                          gallery_labels=attack_IR152A_labels_RESNET_152A,k=1)


# calculate accuracy for RESNET_152C

IR_152A_attack_top1_accuracy_RESNET152C = get_accuracy(probe_features=all_attack_probe_features_RESNET_152C,
                                               probe_labels=all_attack_probe_labels_RESNET_152C,
                                          gallery_features=attack_IR152A_features_RESNET_152C,
                                          gallery_labels=attack_IR152A_labels_RESNET_152C,k=1)



100%|██████████| 35/35 [00:00<00:00, 95.84it/s]


In [ ]:
print("The top1 Accuracy on IR_152A_attacked identity images for IR_152A is: {}%".format(IR_152A_attack_top1_accuracy_IR152A *100))
print("The top1 Accuracy on IR_152A_attacked identity images for IR_152C is: {}%".format(IR_152A_attack_top1_accuracy_IR152C *100))
print("The top1 Accuracy on IR_152A_attacked identity images for RESNET_152A is: {}%".format(IR_152A_attack_top1_accuracy_RESNET152A *100))
print("The top1 Accuracy on IR_152A_attacked identity images for RESNET_152C is: {}%".format(IR_152A_attack_top1_accuracy_RESNET152C *100))


The top1 Accuracy on IR_152A_attacked identity images for IR_152A is: 0.0%
The top1 Accuracy on IR_152A_attacked identity images for IR_152C is: 0.0%
The top1 Accuracy on IR_152A_attacked identity images for RESNET_152A is: 0.0%
The top1 Accuracy on IR_152A_attacked identity images for RESNET_152C is: 0.0%


In [ ]:
# Lets calculate top1 accuracy
# now lets calculate the lowkey accuracies
# calculate accuracy for IR_152A
IR_152A_attack_top5_accuracy_IR152A = get_accuracy(probe_features=all_attack_probe_features_IR_152A,
                                               probe_labels=all_attack_probe_labels_IR_152A,
                                          gallery_features=attack_IR152A_features_IR_152A,
                                          gallery_labels=attack_IR152A_labels_IR_152A,k=5)

# calculate accuracy for IR_152C
IR_152A_attack_top5_accuracy_IR152C = get_accuracy(probe_features=all_attack_probe_features_IR_152C,
                                               probe_labels=all_attack_probe_labels_IR_152C,
                                          gallery_features=attack_IR152A_features_IR_152C,
                                          gallery_labels=attack_IR152A_labels_IR_152C,k=5)

# calculate accuracy for RESNET_152C
IR_152A_attack_top5_accuracy_RESNET152A = get_accuracy(probe_features=all_attack_probe_features_RESNET_152A,
                                               probe_labels=all_attack_probe_labels_RESNET_152A,
                                          gallery_features=attack_IR152A_features_RESNET_152A,
                                          gallery_labels=attack_IR152A_labels_RESNET_152A,k=5)


# calculate accuracy for RESNET_152C

IR_152A_attack_top5_accuracy_RESNET152C = get_accuracy(probe_features=all_attack_probe_features_RESNET_152C,
                                               probe_labels=all_attack_probe_labels_RESNET_152C,
                                          gallery_features=attack_IR152A_features_RESNET_152C,
                                          gallery_labels=attack_IR152A_labels_RESNET_152C,k=5)



100%|██████████| 35/35 [00:00<00:00, 101.59it/s]


In [ ]:
print("The top5 Accuracy on IR_152A_attacked identity images for IR_152A is: {}%".format(IR_152A_attack_top5_accuracy_IR152A *100))
print("The top5 Accuracy on IR_152A_attacked identity images for IR_152C is: {}%".format(IR_152A_attack_top5_accuracy_IR152C *100))
print("The top5 Accuracy on IR_152A_attacked identity images for RESNET_152A is: {}%".format(IR_152A_attack_top5_accuracy_RESNET152A *100))
print("The top5 Accuracy on IR_152A_attacked identity images for RESNET_152C is: {}%".format(IR_152A_attack_top5_accuracy_RESNET152C *100))


The top5 Accuracy on IR_152A_attacked identity images for IR_152A is: 2.857142857142857%
The top5 Accuracy on IR_152A_attacked identity images for IR_152C is: 0.0%
The top5 Accuracy on IR_152A_attacked identity images for RESNET_152A is: 0.0%
The top5 Accuracy on IR_152A_attacked identity images for RESNET_152C is: 5.714285714285714%


### 7. Now lets do attack with IR_152 CosFace for 5 directories and compute top1 and top5

---



In [ ]:
# Now lets do attack on IR_152 C for 5 directories
IR_152C_attack_id_1 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Adam_Brody"
IR_152C_attack_id_2 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Alfred_Molina"
IR_152C_attack_id_3 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Andy_Garcia"
IR_152C_attack_id_4 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Andy_Serkis"
IR_152C_attack_id_5 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Ben_Affleck"



In [ ]:
# protect_dir(dirs_root=IR_152C_attack_id_1,model=model_IR_152C_list)

  0%|          | 0/69 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Adam_Brody/Adam_Brody_592_371.jpeg
Finding reference points


/content/drive/MyDrive/Lowkey code/supp_material/align/first_stage.py:32: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img = Variable(torch.FloatTensor(_preprocess(img)), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:81: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img_boxes = Variable(torch.FloatTensor(img_boxes), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:115: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:102: UserWarning: volatile was remov

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Adam_Brody/Adam_Brody_391_254.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 99%|█████████▊| 68/69 [11:09<00:08,  8.47s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Adam_Brody/Adam_Brody_398_260.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 69/69 [11:17<00:00,  9.82s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=IR_152C_attack_id_2,model=model_IR_152C_list)

  0%|          | 0/43 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Alfred_Molina/Alfred_Molina_4203_2534.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  2%|▏         | 1/43 [00:08<05:54,  8.43s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Alfred_Molina/Alfred_Molina_4271_2575.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  5%|▍         | 2/43 [00:17<05:51,  8.56s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Alfred_Molina/Alfred_Molina_4109_2483.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 44%|████▍     | 19/43 [03:02<03:57,  9.91s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Alfred_Molina/Alfred_Molina_3996_2406.jpeg
Finding reference points


 44%|████▍     | 19/43 [03:03<03:51,  9.64s/it]


IndexError: ignored

In [ ]:
# protect_dir(dirs_root=IR_152C_attack_id_2,model=model_IR_152C_list)

  0%|          | 0/23 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Alfred_Molina/Alfred_Molina_4028_2429.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  4%|▍         | 1/23 [00:08<03:09,  8.61s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Alfred_Molina/Alfred_Molina_4191_2525.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  9%|▊         | 2/23 [00:17<03:07,  8.94s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Alfred_Molina/Alfred_Molina_3940_2362.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 96%|█████████▌| 22/23 [03:46<00:09,  9.84s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Alfred_Molina/Alfred_Molina_3971_2387.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 23/23 [03:56<00:00, 10.29s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=IR_152C_attack_id_3,model=model_IR_152C_list)

  0%|          | 0/68 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Andy_Garcia/Andy_Garcia_5193_2976.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  1%|▏         | 1/68 [00:12<13:58, 12.52s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Andy_Garcia/Andy_Garcia_5226_3001.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  3%|▎         | 2/68 [00:22<12:18, 11.19s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Andy_Garcia/Andy_Garcia_4961_2791.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 99%|█████████▊| 67/68 [11:22<00:10, 10.36s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Andy_Garcia/Andy_Garcia_4960_2790.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 68/68 [11:31<00:00, 10.16s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=IR_152C_attack_id_4,model=model_IR_152C_list)

  0%|          | 0/50 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Andy_Serkis/Andy_Serkis_5896_3424.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  2%|▏         | 1/50 [00:13<11:11, 13.71s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Andy_Serkis/Andy_Serkis_5904_3430.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  4%|▍         | 2/50 [00:25<10:13, 12.78s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Andy_Serkis/Andy_Serkis_5677_3304.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 98%|█████████▊| 49/50 [08:22<00:09,  9.62s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Andy_Serkis/Andy_Serkis_5639_3277.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 50/50 [08:31<00:00, 10.24s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=IR_152C_attack_id_5,model=model_IR_152C_list)

  0%|          | 0/64 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Ben_Affleck/Ben_Affleck_8812_4907.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  2%|▏         | 1/64 [00:09<10:16,  9.78s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Ben_Affleck/Ben_Affleck_8794_4894.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  3%|▎         | 2/64 [00:22<12:07, 11.73s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Ben_Affleck/Ben_Affleck_8455_4656.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 98%|█████████▊| 63/64 [10:51<00:10, 10.05s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery/Ben_Affleck/Ben_Affleck_8476_4672.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 64/64 [11:00<00:00, 10.32s/it]

attack Executed


In [ ]:
# Now lets define a loader for IR_152_ C attacked gallery
IR_152C_attack_gallery_root = '/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery' #90% of validation data
# Generate Dataset
IR_152C_attack_gallery_dataset = datasets.ImageFolder(IR_152C_attack_gallery_root, transform)

# Generate loader with batch size = 16
IR_152C_attack_gallery_loader = torch.utils.data.DataLoader(IR_152C_attack_gallery_dataset, batch_size = 16, shuffle = False, pin_memory = True, num_workers = 0)


In [ ]:
#Lets generate the features for IR_152C_attacked identities
from matplotlib.patches import Arc
#Lets extract features for IR_152A
attack_IR152C_labels_IR_152A,attack_IR152C_images_IR_152A,attack_IR152C_features_IR_152A = extract_feature_face(loader=IR_152C_attack_gallery_loader,model=IR_152A,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__labels_IR_152A.npy", attack_IR152C_labels_IR_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__images_IR_152A.npy", attack_IR152C_images_IR_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__features_IR_152A.npy", attack_IR152C_features_IR_152A)

#Lets extract features for IR_152C
attack_IR152C_labels_IR_152C,attack_IR152C_images_IR_152C,attack_IR152C_features_IR_152C = extract_feature_face(loader=IR_152C_attack_gallery_loader,model=IR_152C,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__labels_IR_152C.npy", attack_IR152C_labels_IR_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__images_IR_152C.npy", attack_IR152C_images_IR_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__features_IR_152C.npy", attack_IR152C_features_IR_152C)

#Lets extract features for RESNET_152A
attack_IR152C_labels_RESNET_152A,attack_IR152C_images_RESNET_152A,attack_IR152C_features_RESNET_152A = extract_feature_face(loader=IR_152C_attack_gallery_loader,model=RESNET_152A,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__labels_RESNET_152A.npy", attack_IR152C_labels_RESNET_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__images_RESNET_152A.npy", attack_IR152C_images_RESNET_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__features_RESNET_152A.npy", attack_IR152C_features_RESNET_152A)

#extract gallery features for RESNET_152C

attack_IR152C_labels_RESNET_152C,attack_IR152C_images_RESNET_152C,attack_IR152C_features_RESNET_152C = extract_feature_face(loader=IR_152C_attack_gallery_loader,model=RESNET_152C,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__labels_RESNET_152C.npy", attack_IR152C_labels_RESNET_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__images_RESNET_152C.npy", attack_IR152C_images_RESNET_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__features_RESNET_152C.npy", attack_IR152C_features_RESNET_152C)




Number of Classes: 25
Features zeros shape: (1243, 512)
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 16
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 32
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 48
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 64
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 80
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 96
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 112
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 128
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The 

In [ ]:
#Lets create the loader for these directories
attack_IR152C_labels_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__labels_IR_152A.npy")
attack_IR152C_features_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__features_IR_152A.npy")

attack_IR152C_labels_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__labels_IR_152C.npy")
attack_IR152C_features_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__features_IR_152C.npy")


attack_IR152C_labels_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__labels_RESNET_152A.npy")
attack_IR152C_features_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__features_RESNET_152A.npy")

attack_IR152C_labels_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__labels_RESNET_152C.npy")
attack_IR152C_features_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/IR_152C_attack/gallery_features/attack_IR152C__features_RESNET_152C.npy")


In [ ]:
# Lets calculate top1 accuracy
# now lets calculate the lowkey accuracies
# calculate accuracy for IR_152A
IR_152C_attack_top1_accuracy_IR152A = get_accuracy(probe_features=all_attack_probe_features_IR_152A,
                                               probe_labels=all_attack_probe_labels_IR_152A,
                                          gallery_features=attack_IR152C_features_IR_152A,
                                          gallery_labels=attack_IR152C_labels_IR_152A,k=1)

# calculate accuracy for IR_152C
IR_152C_attack_top1_accuracy_IR152C = get_accuracy(probe_features=all_attack_probe_features_IR_152C,
                                               probe_labels=all_attack_probe_labels_IR_152C,
                                          gallery_features=attack_IR152C_features_IR_152C,
                                          gallery_labels=attack_IR152C_labels_IR_152C,k=1)

# calculate accuracy for RESNET_152C
IR_152C_attack_top1_accuracy_RESNET152A = get_accuracy(probe_features=all_attack_probe_features_RESNET_152A,
                                               probe_labels=all_attack_probe_labels_RESNET_152A,
                                          gallery_features=attack_IR152C_features_RESNET_152A,
                                          gallery_labels=attack_IR152C_labels_RESNET_152A,k=1)


# calculate accuracy for RESNET_152C

IR_152C_attack_top1_accuracy_RESNET152C = get_accuracy(probe_features=all_attack_probe_features_RESNET_152C,
                                               probe_labels=all_attack_probe_labels_RESNET_152C,
                                          gallery_features=attack_IR152C_features_RESNET_152C,
                                          gallery_labels=attack_IR152C_labels_RESNET_152C,k=1)



100%|██████████| 35/35 [00:00<00:00, 94.87it/s]


In [ ]:
# Lets calculate top1 accuracy
# now lets calculate the lowkey accuracies
# calculate accuracy for IR_152A
IR_152C_attack_top5_accuracy_IR152A = get_accuracy(probe_features=all_attack_probe_features_IR_152A,
                                               probe_labels=all_attack_probe_labels_IR_152A,
                                          gallery_features=attack_IR152C_features_IR_152A,
                                          gallery_labels=attack_IR152C_labels_IR_152A,k=5)

# calculate accuracy for IR_152C
IR_152C_attack_top5_accuracy_IR152C = get_accuracy(probe_features=all_attack_probe_features_IR_152C,
                                               probe_labels=all_attack_probe_labels_IR_152C,
                                          gallery_features=attack_IR152C_features_IR_152C,
                                          gallery_labels=attack_IR152C_labels_IR_152C,k=5)

# calculate accuracy for RESNET_152C
IR_152C_attack_top5_accuracy_RESNET152A = get_accuracy(probe_features=all_attack_probe_features_RESNET_152A,
                                               probe_labels=all_attack_probe_labels_RESNET_152A,
                                          gallery_features=attack_IR152C_features_RESNET_152A,
                                          gallery_labels=attack_IR152C_labels_RESNET_152A,k=5)


# calculate accuracy for RESNET_152C

IR_152C_attack_top5_accuracy_RESNET152C = get_accuracy(probe_features=all_attack_probe_features_RESNET_152C,
                                               probe_labels=all_attack_probe_labels_RESNET_152C,
                                          gallery_features=attack_IR152C_features_RESNET_152C,
                                          gallery_labels=attack_IR152C_labels_RESNET_152C,k=5)



100%|██████████| 35/35 [00:00<00:00, 100.44it/s]


In [ ]:
print("The top1 Accuracy on IR_152C_attacked identity images for IR_152A is: {}%".format(IR_152C_attack_top1_accuracy_IR152A *100))
print("The top1 Accuracy on IR_152C_attacked identity images for IR_152C is: {}%".format(IR_152C_attack_top1_accuracy_IR152C *100))
print("The top1 Accuracy on IR_152C_attacked identity images for RESNET_152A is: {}%".format(IR_152C_attack_top1_accuracy_RESNET152A *100))
print("The top1 Accuracy on IR_152C_attacked identity images for RESNET_152C is: {}%".format(IR_152C_attack_top1_accuracy_RESNET152C *100))


The top1 Accuracy on IR_152C_attacked identity images for IR_152A is: 0.0%
The top1 Accuracy on IR_152C_attacked identity images for IR_152C is: 0.0%
The top1 Accuracy on IR_152C_attacked identity images for RESNET_152A is: 2.857142857142857%
The top1 Accuracy on IR_152C_attacked identity images for RESNET_152C is: 0.0%


In [ ]:
print("The top5 Accuracy on IR_152C_attacked identity images for IR_152A is: {}%".format(IR_152C_attack_top5_accuracy_IR152A *100))
print("The top5 Accuracy on IR_152C_attacked identity images for IR_152C is: {}%".format(IR_152C_attack_top5_accuracy_IR152C *100))
print("The top5 Accuracy on IR_152C_attacked identity images for RESNET_152A is: {}%".format(IR_152C_attack_top5_accuracy_RESNET152A *100))
print("The top5 Accuracy on IR_152C_attacked identity images for RESNET_152C is: {}%".format(IR_152C_attack_top5_accuracy_RESNET152C *100))


The top5 Accuracy on IR_152C_attacked identity images for IR_152A is: 0.0%
The top5 Accuracy on IR_152C_attacked identity images for IR_152C is: 2.857142857142857%
The top5 Accuracy on IR_152C_attacked identity images for RESNET_152A is: 11.428571428571429%
The top5 Accuracy on IR_152C_attacked identity images for RESNET_152C is: 11.428571428571429%


### 8. Now Lets Perform attack with RESNET_152 ArcFace Model and compute top1 and top5 accuracy


---






In [ ]:
# Now lets attack with RESNETArcFace_152 Model
# Now lets do attack on IR_152 C for 5 directories
RESNET_152A_attack_id_1 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Adam_Brody"
RESNET_152A_attack_id_2 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Alfred_Molina"
RESNET_152A_attack_id_3 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Andy_Garcia"
RESNET_152A_attack_id_4 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Andy_Serkis"
RESNET_152A_attack_id_5 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Ben_Affleck"



In [ ]:
# protect_dir(dirs_root=RESNET_152A_attack_id_1,model=model_RESNET_152A_list)

  0%|          | 0/69 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Adam_Brody/Adam_Brody_686_421.jpeg
Finding reference points


/content/drive/MyDrive/Lowkey code/supp_material/align/first_stage.py:32: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img = Variable(torch.FloatTensor(_preprocess(img)), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:81: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img_boxes = Variable(torch.FloatTensor(img_boxes), volatile = True)
/content/drive/MyDrive/Lowkey code/supp_material/align/get_nets.py:115: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(a)
/content/drive/MyDrive/Lowkey code/supp_material/align/detector.py:102: UserWarning: volatile was remov

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Adam_Brody/Adam_Brody_426_281.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 99%|█████████▊| 68/69 [13:24<00:11, 11.66s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Adam_Brody/Adam_Brody_486_317.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 69/69 [13:37<00:00, 11.84s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=RESNET_152A_attack_id_2,model=model_RESNET_152A_list)

  0%|          | 0/42 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Alfred_Molina/Alfred_Molina_4203_2534.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  2%|▏         | 1/42 [00:11<08:01, 11.73s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Alfred_Molina/Alfred_Molina_4271_2575.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  5%|▍         | 2/42 [00:23<07:49, 11.74s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Alfred_Molina/Alfred_Molina_3957_2376.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 98%|█████████▊| 41/42 [08:09<00:11, 11.91s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Alfred_Molina/Alfred_Molina_3940_2362.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 42/42 [08:21<00:00, 11.94s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=RESNET_152A_attack_id_3,model=model_RESNET_152A_list)

  0%|          | 0/68 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Andy_Garcia/Andy_Garcia_5249_3019.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  1%|▏         | 1/68 [00:11<13:16, 11.89s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Andy_Garcia/Andy_Garcia_5226_3001.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  3%|▎         | 2/68 [00:23<13:00, 11.82s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Andy_Garcia/Andy_Garcia_4960_2790.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 99%|█████████▊| 67/68 [13:08<00:11, 11.64s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Andy_Garcia/Andy_Garcia_4962_2792.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 68/68 [13:19<00:00, 11.76s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=RESNET_152A_attack_id_4,model=model_RESNET_152A_list)

  0%|          | 0/50 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Andy_Serkis/Andy_Serkis_5880_3415.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  2%|▏         | 1/50 [00:11<09:20, 11.44s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Andy_Serkis/Andy_Serkis_5911_3435.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  4%|▍         | 2/50 [00:24<09:41, 12.11s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Andy_Serkis/Andy_Serkis_5666_3296.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 98%|█████████▊| 49/50 [09:35<00:12, 12.05s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Andy_Serkis/Andy_Serkis_5635_3274.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 50/50 [09:47<00:00, 11.75s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=RESNET_152A_attack_id_5,model=model_RESNET_152A_list)

  0%|          | 0/64 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Ben_Affleck/Ben_Affleck_8782_4885.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  2%|▏         | 1/64 [00:11<12:18, 11.72s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Ben_Affleck/Ben_Affleck_8794_4894.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  3%|▎         | 2/64 [00:23<12:06, 11.72s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Ben_Affleck/Ben_Affleck_8455_4656.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 98%|█████████▊| 63/64 [12:20<00:11, 11.62s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery/Ben_Affleck/Ben_Affleck_8476_4672.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 64/64 [12:32<00:00, 11.76s/it]

attack Executed


In [ ]:
# Now lets define a loader for RESNET_152_A attacked gallery
RESNET_152A_attack_gallery_root = '/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery' #90% of validation data
# Generate Dataset
RESNET_152A_attack_gallery_dataset = datasets.ImageFolder(RESNET_152A_attack_gallery_root, transform)

# Generate loader with batch size = 16
RESNET_152A_attack_gallery_loader = torch.utils.data.DataLoader(RESNET_152A_attack_gallery_dataset, batch_size = 16, shuffle = False, pin_memory = True, num_workers = 0)


In [ ]:
#Lets generate the features for IR_152C_attacked identities
from matplotlib.patches import Arc
#Lets extract features for IR_152A
attack_RESNET152A_labels_IR_152A,attack_RESNET152A_images_IR_152A,attack_RESNET152A_features_IR_152A = extract_feature_face(loader=RESNET_152A_attack_gallery_loader,model=IR_152A,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__labels_IR_152A.npy", attack_RESNET152A_labels_IR_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__images_IR_152A.npy", attack_RESNET152A_images_IR_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__features_IR_152A.npy", attack_RESNET152A_features_IR_152A)

#Lets extract features for IR_152C
attack_RESNET152A_labels_IR_152C,attack_RESNET152A_images_IR_152C,attack_RESNET152A_features_IR_152C = extract_feature_face(loader=RESNET_152A_attack_gallery_loader,model=IR_152C,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__labels_IR_152C.npy", attack_RESNET152A_labels_IR_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__images_IR_152C.npy", attack_RESNET152A_images_IR_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__features_IR_152C.npy", attack_RESNET152A_features_IR_152C)

#Lets extract features for RESNET_152A
attack_RESNET152A_labels_RESNET_152A,attack_RESNET152A_images_RESNET_152A,attack_RESNET152A_features_RESNET_152A = extract_feature_face(loader=RESNET_152A_attack_gallery_loader,model=RESNET_152A,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__labels_RESNET_152A.npy", attack_RESNET152A_labels_RESNET_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__images_RESNET_152A.npy", attack_RESNET152A_images_RESNET_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__features_RESNET_152A.npy", attack_RESNET152A_features_RESNET_152A)

#extract gallery features for RESNET_152C

attack_RESNET152A_labels_RESNET_152C,attack_RESNET152A_images_RESNET_152C,attack_RESNET152A_features_RESNET_152C = extract_feature_face(loader=RESNET_152A_attack_gallery_loader,model=RESNET_152C,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__labels_RESNET_152C.npy", attack_RESNET152A_labels_RESNET_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__images_RESNET_152C.npy", attack_RESNET152A_images_RESNET_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__features_RESNET_152C.npy", attack_RESNET152A_features_RESNET_152C)




Number of Classes: 24
Features zeros shape: (1243, 512)
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 16
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 32
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 48
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 64
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 80
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 96
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 112
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The value of idx is : 128
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1243, 512)
The 

In [ ]:
#Lets create the loader for these directories
attack_RESNET152A_labels_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__labels_IR_152A.npy")
attack_RESNET152A_features_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__features_IR_152A.npy")

attack_RESNET152A_labels_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__labels_IR_152C.npy")
attack_RESNET152A_features_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__features_IR_152C.npy")


attack_RESNET152A_labels_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__labels_RESNET_152A.npy")
attack_RESNET152A_features_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__features_RESNET_152A.npy")

attack_RESNET152A_labels_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__labels_RESNET_152C.npy")
attack_RESNET152A_features_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152A_attack/gallery_features/attack_RESNET152A__features_RESNET_152C.npy")


In [ ]:
# Lets calculate top1 accuracy
# now lets calculate the lowkey accuracies
# calculate accuracy for IR_152A
RESNET_152A_attack_top1_accuracy_IR152A = get_accuracy(probe_features=all_attack_probe_features_IR_152A,
                                               probe_labels=all_attack_probe_labels_IR_152A,
                                          gallery_features=attack_RESNET152A_features_IR_152A,
                                          gallery_labels=attack_RESNET152A_labels_IR_152A,k=1)

# calculate accuracy for IR_152C
RESNET_152A_attack_top1_accuracy_IR152C = get_accuracy(probe_features=all_attack_probe_features_IR_152C,
                                               probe_labels=all_attack_probe_labels_IR_152C,
                                          gallery_features=attack_RESNET152A_features_IR_152C,
                                          gallery_labels=attack_RESNET152A_labels_IR_152C,k=1)

# calculate accuracy for RESNET_152C
RESNET_152A_attack_top1_accuracy_RESNET152A = get_accuracy(probe_features=all_attack_probe_features_RESNET_152A,
                                               probe_labels=all_attack_probe_labels_RESNET_152A,
                                          gallery_features=attack_RESNET152A_features_RESNET_152A,
                                          gallery_labels=attack_RESNET152A_labels_RESNET_152A,k=1)


# calculate accuracy for RESNET_152C

RESNET_152A_attack_top1_accuracy_RESNET152C = get_accuracy(probe_features=all_attack_probe_features_RESNET_152C,
                                               probe_labels=all_attack_probe_labels_RESNET_152C,
                                          gallery_features=attack_RESNET152A_features_RESNET_152C,
                                          gallery_labels=attack_RESNET152A_labels_RESNET_152C,k=1)



100%|██████████| 35/35 [00:00<00:00, 101.30it/s]


In [ ]:
# Lets calculate top1 accuracy
# now lets calculate the lowkey accuracies
# calculate accuracy for IR_152A
RESNET_152A_attack_top5_accuracy_IR152A = get_accuracy(probe_features=all_attack_probe_features_IR_152A,
                                               probe_labels=all_attack_probe_labels_IR_152A,
                                          gallery_features=attack_RESNET152A_features_IR_152A,
                                          gallery_labels=attack_RESNET152A_labels_IR_152A,k=5)

# calculate accuracy for IR_152C
RESNET_152A_attack_top5_accuracy_IR152C = get_accuracy(probe_features=all_attack_probe_features_IR_152C,
                                               probe_labels=all_attack_probe_labels_IR_152C,
                                          gallery_features=attack_RESNET152A_features_IR_152C,
                                          gallery_labels=attack_RESNET152A_labels_IR_152C,k=5)

# calculate accuracy for RESNET_152C
RESNET_152A_attack_top5_accuracy_RESNET152A = get_accuracy(probe_features=all_attack_probe_features_RESNET_152A,
                                               probe_labels=all_attack_probe_labels_RESNET_152A,
                                          gallery_features=attack_RESNET152A_features_RESNET_152A,
                                          gallery_labels=attack_RESNET152A_labels_RESNET_152A,k=5)


# calculate accuracy for RESNET_152C

RESNET_152A_attack_top5_accuracy_RESNET152C = get_accuracy(probe_features=all_attack_probe_features_RESNET_152C,
                                               probe_labels=all_attack_probe_labels_RESNET_152C,
                                          gallery_features=attack_RESNET152A_features_RESNET_152C,
                                          gallery_labels=attack_RESNET152A_labels_RESNET_152C,k=5)



100%|██████████| 35/35 [00:00<00:00, 95.92it/s]


In [ ]:
print("The top1 Accuracy on RESNET_152A_attacked identity images for IR_152A is: {}%".format(RESNET_152A_attack_top1_accuracy_IR152A *100))
print("The top1 Accuracy on RESNET_152A_attacked identity images for IR_152C is: {}%".format(RESNET_152A_attack_top1_accuracy_IR152C *100))
print("The top1 Accuracy on RESNET_152A_attacked identity images for RESNET_152A is: {}%".format(RESNET_152A_attack_top1_accuracy_RESNET152A *100))
print("The top1 Accuracy on RESNET_152A_attacked identity images for RESNET_152C is: {}%".format(RESNET_152A_attack_top1_accuracy_RESNET152C *100))


The top1 Accuracy on RESNET_152A_attacked identity images for IR_152A is: 0.0%
The top1 Accuracy on RESNET_152A_attacked identity images for IR_152C is: 0.0%
The top1 Accuracy on RESNET_152A_attacked identity images for RESNET_152A is: 0.0%
The top1 Accuracy on RESNET_152A_attacked identity images for RESNET_152C is: 0.0%


In [ ]:
print("The top5 Accuracy on RESNET_152A_attacked identity images for IR_152A is: {}%".format(RESNET_152A_attack_top5_accuracy_IR152A *100))
print("The top5 Accuracy on RESNET_152A_attacked identity images for IR_152C is: {}%".format(RESNET_152A_attack_top5_accuracy_IR152C *100))
print("The top5 Accuracy on RESNET_152A_attacked identity images for RESNET_152A is: {}%".format(RESNET_152A_attack_top5_accuracy_RESNET152A *100))
print("The top5 Accuracy on RESNET_152A_attacked identity images for RESNET_152C is: {}%".format(RESNET_152A_attack_top5_accuracy_RESNET152C *100))


The top5 Accuracy on RESNET_152A_attacked identity images for IR_152A is: 0.0%
The top5 Accuracy on RESNET_152A_attacked identity images for IR_152C is: 2.857142857142857%
The top5 Accuracy on RESNET_152A_attacked identity images for RESNET_152A is: 14.285714285714285%
The top5 Accuracy on RESNET_152A_attacked identity images for RESNET_152C is: 8.571428571428571%


In [ ]:

tableEvaluation4 = PrettyTable()

# Add data for table display

tableEvaluation4.title = 'Evaluation pipeline Top 1'
tableEvaluation4.field_names = ['Attacker','IR_152A', 'IR_152C','RESNET_152A','RESNET_152C']
tableEvaluation4.add_row(['clean_images_Feature_1', clean_top1_accuracy_IR152A*100,clean_top1_accuracy_IR152C*100,clean_top1_accuracy_RESNET152A*100,clean_top1_accuracy_RESNET152C*100])
tableEvaluation4.add_row(['Ensemble_attack_Feature_1',adversial_top1_accuracy_IR152A*100,adversial_top1_accuracy_IR152C*100, adversial_top1_accuracy_RESNET152A*100,adversial_top1_accuracy_RESNET152C*100])
tableEvaluation4.add_row(['Lowkey_ensemble_attack_5_directories',lowkey_top1_accuracy_IR152A *100,lowkey_top1_accuracy_IR152C *100, lowkey_top1_accuracy_RESNET152A *100,lowkey_top1_accuracy_RESNET152A *100])
tableEvaluation4.add_row(['IR_152_ArcFace_attack_5_directories',IR_152A_attack_top1_accuracy_IR152A *100,IR_152A_attack_top1_accuracy_IR152C *100, IR_152A_attack_top1_accuracy_RESNET152A *100,IR_152A_attack_top1_accuracy_RESNET152C *100])
tableEvaluation4.add_row(['IR_152_CosFace_attack_5_directories',IR_152C_attack_top1_accuracy_IR152A *100,IR_152C_attack_top1_accuracy_IR152C *100, IR_152C_attack_top1_accuracy_RESNET152A *100,IR_152C_attack_top1_accuracy_RESNET152C *100])
tableEvaluation4.add_row(['RESNET_152_ArcFace_attack_5_directories',RESNET_152A_attack_top1_accuracy_IR152A *100,RESNET_152A_attack_top1_accuracy_IR152C *100, RESNET_152A_attack_top1_accuracy_RESNET152A *100,RESNET_152A_attack_top1_accuracy_RESNET152C *100])

print(tableEvaluation4)


+-------------------------------------------------------------------------------------------------------------------------+
|                                                Evaluation pipeline Top 1                                                |
+-----------------------------------------+-------------------+-------------------+-------------------+-------------------+
|                 Attacker                |      IR_152A      |      IR_152C      |    RESNET_152A    |    RESNET_152C    |
+-----------------------------------------+-------------------+-------------------+-------------------+-------------------+
|          clean_images_Feature_1         | 99.33774834437085 | 99.33774834437085 | 99.33774834437085 | 99.33774834437085 |
|        Ensemble_attack_Feature_1        | 78.80794701986756 | 79.47019867549669 | 79.47019867549669 | 79.47019867549669 |
|   Lowkey_ensemble_attack_5_directories  |        0.0        |        0.0        |        0.0        |        0.0        |
|   IR_1

In [ ]:

tableEvaluation3 = PrettyTable()

# Add data for table display

tableEvaluation3.title = 'Evaluation pipeline Top 5'
tableEvaluation3.field_names = ['Attacker','IR_152A', 'IR_152C','RESNET_152A','RESNET_152C']
tableEvaluation4.add_row(['clean_images_Feature_1', clean_top1_accuracy_IR152A*100,clean_top1_accuracy_IR152C*100,clean_top1_accuracy_RESNET152A*100,clean_top1_accuracy_RESNET152C*100])
tableEvaluation4.add_row(['Ensemble_attack_Feature_1',adversial_top1_accuracy_IR152A*100,adversial_top1_accuracy_IR152C*100, adversial_top1_accuracy_RESNET152A*100,adversial_top1_accuracy_RESNET152C*100])
tableEvaluation3.add_row(['Lowkey_ensemble_attack_5_directories',lowkey_top5_accuracy_IR152A *100,lowkey_top5_accuracy_IR152C *100, lowkey_top5_accuracy_RESNET152A *100,lowkey_top5_accuracy_RESNET152A *100])
tableEvaluation3.add_row(['IR_152_ArcFace_attack_5_directories',IR_152A_attack_top5_accuracy_IR152A *100,IR_152A_attack_top5_accuracy_IR152C *100, IR_152A_attack_top5_accuracy_RESNET152A *100,IR_152A_attack_top5_accuracy_RESNET152C *100])
tableEvaluation3.add_row(['IR_152_CosFace_attack_5_directories',IR_152C_attack_top5_accuracy_IR152A *100,IR_152C_attack_top5_accuracy_IR152C *100, IR_152C_attack_top5_accuracy_RESNET152A *100,IR_152C_attack_top5_accuracy_RESNET152C *100])
tableEvaluation3.add_row(['RESNET_152_ArcFace_attack_5_directories',RESNET_152A_attack_top5_accuracy_IR152A *100,RESNET_152A_attack_top5_accuracy_IR152C *100, RESNET_152A_attack_top5_accuracy_RESNET152A *100,RESNET_152A_attack_top5_accuracy_RESNET152C *100])

print(tableEvaluation3)


+---------------------------------------------------------------------------------------------------------------------------+
|                                                 Evaluation pipeline Top 5                                                 |
+-----------------------------------------+-------------------+-------------------+--------------------+--------------------+
|                 Attacker                |      IR_152A      |      IR_152C      |    RESNET_152A     |    RESNET_152C     |
+-----------------------------------------+-------------------+-------------------+--------------------+--------------------+
|   Lowkey_ensemble_attack_5_directories  | 2.857142857142857 | 5.714285714285714 | 14.285714285714285 | 14.285714285714285 |
|   IR_152_ArcFace_attack_5_directories   | 2.857142857142857 |        0.0        |        0.0         | 5.714285714285714  |
|   IR_152_CosFace_attack_5_directories   |        0.0        | 2.857142857142857 | 11.428571428571429 | 11.4285714285

### 9. Now finally Performing attack with RESNET_152C Model and calculating top 1 and top5 accuracy

---



In [ ]:
################Now lets do it finally for RESNET_152C
# Now lets attack with RESNETCosFace_152 Model
RESNET_152C_attack_id_1 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Adam_Brody"
RESNET_152C_attack_id_2 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Alfred_Molina"
RESNET_152C_attack_id_3 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Andy_Garcia"
RESNET_152C_attack_id_4 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Andy_Serkis"
RESNET_152C_attack_id_5 = "/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Ben_Affleck"



In [ ]:
# protect_dir(dirs_root=RESNET_152C_attack_id_1,model=model_RESNET_152C_list)

  0%|          | 0/69 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Adam_Brody/Adam_Brody_592_371.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  1%|▏         | 1/69 [00:12<14:20, 12.66s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Adam_Brody/Adam_Brody_659_409.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  3%|▎         | 2/69 [00:24<13:20, 11.95s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Adam_Brody/Adam_Brody_375_247.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 99%|█████████▊| 68/69 [13:11<00:11, 11.68s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Adam_Brody/Adam_Brody_398_260.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 69/69 [13:23<00:00, 11.64s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=RESNET_152C_attack_id_2,model=model_RESNET_152C_list)

  0%|          | 0/42 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Alfred_Molina/Alfred_Molina_4201_2532.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  2%|▏         | 1/42 [00:11<07:56, 11.63s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Alfred_Molina/Alfred_Molina_4271_2575.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  5%|▍         | 2/42 [00:23<07:45, 11.65s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Alfred_Molina/Alfred_Molina_3971_2387.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 98%|█████████▊| 41/42 [07:59<00:11, 11.64s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Alfred_Molina/Alfred_Molina_3938_2360.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 42/42 [08:11<00:00, 11.70s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=RESNET_152C_attack_id_3,model=model_RESNET_152C_list)

  0%|          | 0/68 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Andy_Garcia/Andy_Garcia_5294_3049.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  1%|▏         | 1/68 [00:11<12:40, 11.35s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Andy_Garcia/Andy_Garcia_5253_3022.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  3%|▎         | 2/68 [00:23<13:13, 12.02s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Andy_Garcia/Andy_Garcia_4960_2790.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 99%|█████████▊| 67/68 [13:12<00:11, 11.65s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Andy_Garcia/Andy_Garcia_4962_2792.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 68/68 [13:23<00:00, 11.82s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=RESNET_152C_attack_id_4,model=model_RESNET_152C_list)

  0%|          | 0/50 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Andy_Serkis/Andy_Serkis_5844_3393.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  2%|▏         | 1/50 [00:11<09:24, 11.52s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Andy_Serkis/Andy_Serkis_5875_3412.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  4%|▍         | 2/50 [00:22<09:11, 11.49s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Andy_Serkis/Andy_Serkis_5656_3290.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 98%|█████████▊| 49/50 [09:27<00:11, 11.72s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Andy_Serkis/Andy_Serkis_5685_3308.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 50/50 [09:38<00:00, 11.57s/it]

attack Executed


In [ ]:
# protect_dir(dirs_root=RESNET_152C_attack_id_5,model=model_RESNET_152C_list)

  0%|          | 0/64 [00:00<?, ?it/s]

The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Ben_Affleck/Ben_Affleck_8812_4907.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  2%|▏         | 1/64 [00:11<12:10, 11.60s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Ben_Affleck/Ben_Affleck_8794_4894.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


  3%|▎         | 2/64 [00:23<11:52, 11.49s/it]

...

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Ben_Affleck/Ben_Affleck_8471_4670.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


 98%|█████████▊| 63/64 [12:15<00:11, 11.85s/it]

attack Executed
The image processed is : /content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery/Ben_Affleck/Ben_Affleck_8476_4672.jpeg
Finding reference points
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 64/64 [12:27<00:00, 11.67s/it]

attack Executed


In [ ]:
# Now lets define a loader for IR_152_attacked gallery
RESNET_152C_attack_gallery_root = '/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery' #90% of validation data
# Generate Dataset
RESNET_152C_attack_gallery_dataset = datasets.ImageFolder(RESNET_152C_attack_gallery_root, transform)

# Generate loader with batch size = 16
RESNET_152C_attack_gallery_loader = torch.utils.data.DataLoader(RESNET_152C_attack_gallery_dataset, batch_size = 16, shuffle = False, pin_memory = True, num_workers = 0)


In [ ]:
#Lets generate the features for IR_152C_attacked identities
from matplotlib.patches import Arc
#Lets extract features for IR_152A
attack_RESNET152C_labels_IR_152A,attack_RESNET152C_images_IR_152A,attack_RESNET152C_features_IR_152A = extract_feature_face(loader=RESNET_152C_attack_gallery_loader,model=IR_152A,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__labels_IR_152A.npy", attack_RESNET152C_labels_IR_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__images_IR_152A.npy", attack_RESNET152C_images_IR_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__features_IR_152A.npy", attack_RESNET152C_features_IR_152A)

#Lets extract features for IR_152C
attack_RESNET152C_labels_IR_152C,attack_RESNET152C_images_IR_152C,attack_RESNET152C_features_IR_152C = extract_feature_face(loader=RESNET_152C_attack_gallery_loader,model=IR_152C,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__labels_IR_152C.npy", attack_RESNET152C_labels_IR_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__images_IR_152C.npy", attack_RESNET152C_images_IR_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__features_IR_152C.npy", attack_RESNET152C_features_IR_152C)

#Lets extract features for RESNET_152A
attack_RESNET152C_labels_RESNET_152A,attack_RESNET152C_images_RESNET_152A,attack_RESNET152C_features_RESNET_152A = extract_feature_face(loader=RESNET_152C_attack_gallery_loader,model=RESNET_152A,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__labels_RESNET_152A.npy", attack_RESNET152C_labels_RESNET_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__images_RESNET_152A.npy", attack_RESNET152C_images_RESNET_152A)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__features_RESNET_152A.npy", attack_RESNET152C_features_RESNET_152A)

#extract gallery features for RESNET_152C

attack_RESNET152C_labels_RESNET_152C,attack_RESNET152C_images_RESNET_152C,attack_RESNET152C_features_RESNET_152C = extract_feature_face(loader=RESNET_152C_attack_gallery_loader,model=RESNET_152C,embedding_size=EMBEDDING_SIZE,device=device)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__labels_RESNET_152C.npy", attack_RESNET152C_labels_RESNET_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__images_RESNET_152C.npy", attack_RESNET152C_images_RESNET_152C)
np.save("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__features_RESNET_152C.npy", attack_RESNET152C_features_RESNET_152C)




Number of Classes: 24
Features zeros shape: (1264, 512)
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1264, 512)
The value of idx is : 16
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1264, 512)
The value of idx is : 32
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1264, 512)
The value of idx is : 48
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1264, 512)
The value of idx is : 64
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1264, 512)
The value of idx is : 80
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1264, 512)
The value of idx is : 96
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1264, 512)
The value of idx is : 112
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1264, 512)
The value of idx is : 128
idx + batch_size <= len(loader.dataset)
Features l2_norm shape else : (1264, 512)
The 

In [ ]:
# #Lets create the loader for these directories
# attack_RESNET152C_labels_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__labels_IR_152A.npy")
# attack_RESNET152C_features_IR_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__features_IR_152A.npy")

# attack_RESNET152C_labels_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__labels_IR_152C.npy")
# attack_RESNET152C_features_IR_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__features_IR_152C.npy")


# attack_RESNET152C_labels_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__labels_RESNET_152A.npy")
# attack_RESNET152C_features_RESNET_152A = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__features_RESNET_152A.npy")

# attack_RESNET152C_labels_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__labels_RESNET_152C.npy")
# attack_RESNET152C_features_RESNET_152C = np.load("/content/drive/MyDrive/Lowkey code/faceScrub/actors/cleanData/dataAlign/adversial_data/RESNET_152C_attack/gallery_features/attack_RESNET152C__features_RESNET_152C.npy")


In [ ]:
# Lets calculate top1 accuracy
# now lets calculate the lowkey accuracies
# calculate accuracy for IR_152A
RESNET_152C_attack_top1_accuracy_IR152A = get_accuracy(probe_features=all_attack_probe_features_IR_152A,
                                               probe_labels=all_attack_probe_labels_IR_152A,
                                          gallery_features=attack_RESNET152C_features_IR_152A,
                                          gallery_labels=attack_RESNET152C_labels_IR_152A,k=1)

# calculate accuracy for IR_152C
RESNET_152C_attack_top1_accuracy_IR152C = get_accuracy(probe_features=all_attack_probe_features_IR_152C,
                                               probe_labels=all_attack_probe_labels_IR_152C,
                                          gallery_features=attack_RESNET152C_features_IR_152C,
                                          gallery_labels=attack_RESNET152C_labels_IR_152C,k=1)

# calculate accuracy for RESNET_152C
RESNET_152C_attack_top1_accuracy_RESNET152A = get_accuracy(probe_features=all_attack_probe_features_RESNET_152A,
                                               probe_labels=all_attack_probe_labels_RESNET_152A,
                                          gallery_features=attack_RESNET152C_features_RESNET_152A,
                                          gallery_labels=attack_RESNET152C_labels_RESNET_152A,k=1)


# calculate accuracy for RESNET_152C

RESNET_152C_attack_top1_accuracy_RESNET152C = get_accuracy(probe_features=all_attack_probe_features_RESNET_152C,
                                               probe_labels=all_attack_probe_labels_RESNET_152C,
                                          gallery_features=attack_RESNET152C_features_RESNET_152C,
                                          gallery_labels=attack_RESNET152C_labels_RESNET_152C,k=1)



100%|██████████| 35/35 [00:00<00:00, 78.23it/s]


In [ ]:
# Lets calculate top1 accuracy
# now lets calculate the lowkey accuracies
# calculate accuracy for IR_152A
RESNET_152C_attack_top5_accuracy_IR152A = get_accuracy(probe_features=all_attack_probe_features_IR_152A,
                                               probe_labels=all_attack_probe_labels_IR_152A,
                                          gallery_features=attack_RESNET152C_features_IR_152A,
                                          gallery_labels=attack_RESNET152C_labels_IR_152A,k=5)

# calculate accuracy for IR_152C
RESNET_152C_attack_top5_accuracy_IR152C = get_accuracy(probe_features=all_attack_probe_features_IR_152C,
                                               probe_labels=all_attack_probe_labels_IR_152C,
                                          gallery_features=attack_RESNET152C_features_IR_152C,
                                          gallery_labels=attack_RESNET152C_labels_IR_152C,k=5)

# calculate accuracy for RESNET_152C
RESNET_152C_attack_top5_accuracy_RESNET152A = get_accuracy(probe_features=all_attack_probe_features_RESNET_152A,
                                               probe_labels=all_attack_probe_labels_RESNET_152A,
                                          gallery_features=attack_RESNET152C_features_RESNET_152A,
                                          gallery_labels=attack_RESNET152C_labels_RESNET_152A,k=5)


# calculate accuracy for RESNET_152C

RESNET_152C_attack_top5_accuracy_RESNET152C = get_accuracy(probe_features=all_attack_probe_features_RESNET_152C,
                                               probe_labels=all_attack_probe_labels_RESNET_152C,
                                          gallery_features=attack_RESNET152C_features_RESNET_152C,
                                          gallery_labels=attack_RESNET152C_labels_RESNET_152C,k=5)



100%|██████████| 35/35 [00:00<00:00, 102.50it/s]


In [ ]:
print("The top1 Accuracy on RESNET_152C_attacked identity images for IR_152A is: {}%".format(RESNET_152C_attack_top1_accuracy_IR152A *100))
print("The top1 Accuracy on RESNET_152C_attacked identity images for IR_152C is: {}%".format(RESNET_152C_attack_top1_accuracy_IR152C *100))
print("The top1 Accuracy on RESNET_152C_attacked identity images for RESNET_152A is: {}%".format(RESNET_152C_attack_top1_accuracy_RESNET152A *100))
print("The top1 Accuracy on RESNET_152C_attacked identity images for RESNET_152C is: {}%".format(RESNET_152C_attack_top1_accuracy_RESNET152C *100))


The top1 Accuracy on RESNET_152C_attacked identity images for IR_152A is: 0.0%
The top1 Accuracy on RESNET_152C_attacked identity images for IR_152C is: 0.0%
The top1 Accuracy on RESNET_152C_attacked identity images for RESNET_152A is: 2.857142857142857%
The top1 Accuracy on RESNET_152C_attacked identity images for RESNET_152C is: 2.857142857142857%


In [ ]:
print("The top5 Accuracy on RESNET_152C_attacked identity images for IR_152A is: {}%".format(RESNET_152C_attack_top5_accuracy_IR152A *100))
print("The top5 Accuracy on RESNET_152C_attacked identity images for IR_152C is: {}%".format(RESNET_152C_attack_top5_accuracy_IR152C *100))
print("The top5 Accuracy on RESNET_152C_attacked identity images for RESNET_152A is: {}%".format(RESNET_152C_attack_top5_accuracy_RESNET152A *100))
print("The top5 Accuracy on RESNET_152C_attacked identity images for RESNET_152C is: {}%".format(RESNET_152C_attack_top5_accuracy_RESNET152C *100))


The top5 Accuracy on RESNET_152C_attacked identity images for IR_152A is: 0.0%
The top5 Accuracy on RESNET_152C_attacked identity images for IR_152C is: 2.857142857142857%
The top5 Accuracy on RESNET_152C_attacked identity images for RESNET_152A is: 11.428571428571429%
The top5 Accuracy on RESNET_152C_attacked identity images for RESNET_152C is: 17.142857142857142%


### 10. Now lets finally print all evaluation results in the table

---



In [ ]:

tableEvaluation5 = PrettyTable()

# Add data for table display

tableEvaluation5.title = 'Evaluation pipeline Top 1 - 5 identities'
tableEvaluation5.field_names = ['Attacker','IR_152A', 'IR_152C','RN_152A','RN_152C']
tableEvaluation5.add_row(['clean_images_all_gallery', round(clean_top1_accuracy_IR152A*100,2),round(clean_top1_accuracy_IR152C*100,2),round(clean_top1_accuracy_RESNET152A*100,2),round(clean_top1_accuracy_RESNET152C*100,2)])
tableEvaluation5.add_row(['Ensemble_attack_all_gallery',round(adversial_top1_accuracy_IR152A*100,2),round(adversial_top1_accuracy_IR152C*100,2),round(adversial_top1_accuracy_RESNET152A*100,2),round(adversial_top1_accuracy_RESNET152C*100,2)])
tableEvaluation5.add_row(['Ensemble_lowkey',round(lowkey_top1_accuracy_IR152A *100,2),round(lowkey_top1_accuracy_IR152C *100,2), round(lowkey_top1_accuracy_RESNET152A *100,2),round(lowkey_top1_accuracy_RESNET152C *100,2)])
tableEvaluation5.add_row(['IR_152A',round(IR_152A_attack_top1_accuracy_IR152A *100,2),round(IR_152A_attack_top1_accuracy_IR152C *100,2), round(IR_152A_attack_top1_accuracy_RESNET152A *100,2),round(IR_152A_attack_top1_accuracy_RESNET152C *100,2)])
tableEvaluation5.add_row(['IR_152C',round(IR_152C_attack_top1_accuracy_IR152A *100,2),round(IR_152C_attack_top1_accuracy_IR152C *100,2), round(IR_152C_attack_top1_accuracy_RESNET152A *100,2),round(IR_152C_attack_top1_accuracy_RESNET152C *100,2)])
tableEvaluation5.add_row(['RN_152A',round(RESNET_152A_attack_top1_accuracy_IR152A *100,2),round(RESNET_152A_attack_top1_accuracy_IR152C *100,2), round(RESNET_152A_attack_top1_accuracy_RESNET152A *100,2),round(RESNET_152A_attack_top1_accuracy_RESNET152C *100,2)])
tableEvaluation5.add_row(['RN_152C',round(RESNET_152C_attack_top1_accuracy_IR152A *100,2),round(RESNET_152C_attack_top1_accuracy_IR152C *100,2), round(RESNET_152C_attack_top1_accuracy_RESNET152A *100,2),round(RESNET_152C_attack_top1_accuracy_RESNET152C *100,2)])

print(tableEvaluation5)


+---------------------------------------------------------------------+
|               Evaluation pipeline Top 1 - 5 identities              |
+-----------------------------+---------+---------+---------+---------+
|           Attacker          | IR_152A | IR_152C | RN_152A | RN_152C |
+-----------------------------+---------+---------+---------+---------+
|   clean_images_all_gallery  |  99.34  |  99.34  |  99.34  |  99.34  |
| Ensemble_attack_all_gallery |  78.81  |  79.47  |  79.47  |  79.47  |
|       Ensemble_lowkey       |   0.0   |   0.0   |   0.0   |   0.0   |
|           IR_152A           |   0.0   |   0.0   |   0.0   |   0.0   |
|           IR_152C           |   0.0   |   0.0   |   2.86  |   0.0   |
|           RN_152A           |   0.0   |   0.0   |   0.0   |   0.0   |
|           RN_152C           |   0.0   |   0.0   |   2.86  |   2.86  |
+-----------------------------+---------+---------+---------+---------+


In [ ]:

tableEvaluation6 = PrettyTable()

# Add data for table display

tableEvaluation6.title = 'Evaluation pipeline Top 5 - 5 identities'
tableEvaluation6.field_names = ['Attacker','IR_152A', 'IR_152C','RN_152A','RN_152C']
tableEvaluation6.add_row(['clean_images_all_gallery', round(clean_top5_accuracy_IR152A*100,2),round(clean_top5_accuracy_IR152C*100,2),round(clean_top5_accuracy_RESNET152A*100,2),round(clean_top5_accuracy_RESNET152C*100,2)])
tableEvaluation6.add_row(['Ensemble_attack_all_gallery',round(adversial_top5_accuracy_IR152A*100,2),round(adversial_top5_accuracy_IR152C*100,2),round(adversial_top5_accuracy_RESNET152A*100,2),round(adversial_top5_accuracy_RESNET152C*100,2)])
tableEvaluation6.add_row(['Ensemble_lowkey',round(lowkey_top5_accuracy_IR152A *100,2),round(lowkey_top5_accuracy_IR152C *100,2), round(lowkey_top5_accuracy_RESNET152A *100,2),round(lowkey_top5_accuracy_RESNET152C *100,2)])
tableEvaluation6.add_row(['IR_152A',round(IR_152A_attack_top5_accuracy_IR152A *100,2),round(IR_152A_attack_top5_accuracy_IR152C *100,2), round(IR_152A_attack_top5_accuracy_RESNET152A *100,2),round(IR_152A_attack_top5_accuracy_RESNET152C *100,2)])
tableEvaluation6.add_row(['IR_152C',round(IR_152C_attack_top5_accuracy_IR152A *100,2),round(IR_152C_attack_top5_accuracy_IR152C *100,2), round(IR_152C_attack_top5_accuracy_RESNET152A *100,2),round(IR_152C_attack_top5_accuracy_RESNET152C *100,2)])
tableEvaluation6.add_row(['RN_152A',round(RESNET_152A_attack_top5_accuracy_IR152A *100,2),round(RESNET_152A_attack_top5_accuracy_IR152C *100,2), round(RESNET_152A_attack_top5_accuracy_RESNET152A *100,2),round(RESNET_152A_attack_top5_accuracy_RESNET152C *100,2)])
tableEvaluation6.add_row(['RN_152C',round(RESNET_152C_attack_top5_accuracy_IR152A *100,2),round(RESNET_152C_attack_top5_accuracy_IR152C *100,2), round(RESNET_152C_attack_top5_accuracy_RESNET152A *100,2),round(RESNET_152C_attack_top5_accuracy_RESNET152C *100,2)])

print(tableEvaluation6)


+---------------------------------------------------------------------+
|               Evaluation pipeline Top 5 - 5 identities              |
+-----------------------------+---------+---------+---------+---------+
|           Attacker          | IR_152A | IR_152C | RN_152A | RN_152C |
+-----------------------------+---------+---------+---------+---------+
|   clean_images_all_gallery  |  99.34  |  99.34  |  99.34  |  100.0  |
| Ensemble_attack_all_gallery |  79.47  |  79.47  |  79.47  |  80.13  |
|       Ensemble_lowkey       |   2.86  |   5.71  |  14.29  |  14.29  |
|           IR_152A           |   2.86  |   0.0   |   0.0   |   5.71  |
|           IR_152C           |   0.0   |   2.86  |  11.43  |  11.43  |
|           RN_152A           |   0.0   |   2.86  |  14.29  |   8.57  |
|           RN_152C           |   0.0   |   2.86  |  11.43  |  17.14  |
+-----------------------------+---------+---------+---------+---------+
